In [530]:
import pandas as pd 
import plotly.express as px
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [531]:
data = pd.read_csv('/Users/sergeymedvedev/Downloads/data6.csv')
data

,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime
0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00
1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00
2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00
3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00
4,T60DAR,PAS,NJ,Pickup,14-No Standing,33,Manhattan,525,175th st,2013-08-11 18:17:00
...,...,...,...,...,...,...,...,...,...,...
33932130,GWK2897,999,NY,Sedan,19-No Stand (bus stop),113,Queens,NaN,terminal 7th arrivals,2016-06-30 04:12:00
33932131,HFF8478,PAS,NY,Suburban,14-No Standing,113,Queens,NaN,terminal 8th arrivals,2016-06-30 06:15:00
33932132,GZY9542,PAS,NY,Sedan,70-Reg. Sticker Missing (NYS),42,Bronx,3400,3rd ave,2016-06-30 06:23:00
33932133,HAB9889,PAS,NY,Suburban,16-No Std (Com Veh) Com Plate,43,Bronx,1489,ave,2016-06-30 18:45:00


In [532]:
data['time'] = pd.to_datetime(data['issue_datetime'])
#make them continiuous 'day_cont', 'week_cont'
data['day'] = pd.to_datetime(data['time']).values.astype('datetime64[D]')
data['week']  = pd.to_datetime(data['time']).values.astype('datetime64[W]')
data['month'] = data['time'].values.astype('datetime64[M]')

# DATA EXPLORATION

# NYC AS A WHOLE

In [533]:
#Which registration state is the most popular in NYC
nyc_reg_state = data.groupby('registration_state')['plate_id'].count().reset_index()
nyc_reg_state.head()

,registration_state,plate_id
0,99,93584
1,AB,1035
2,AK,3432
3,AL,23988
4,AR,10892


In [534]:
fig = px.bar(nyc_reg_state, 
             x="registration_state", 
             y="plate_id",
             labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             },
             title="Number of violations in NYC by vehicle registration state")
fig.show()

76.92% of the total violations in NYC came from locals from NY

In [535]:
#Filter dataframe to contain only registration plates from NY
data_ny = data[data['registration_state'] == 'NY']

In [536]:
#Which plate types are the most popular in NYC
data_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,PAS,16451679
1,COM,7406744
2,OMT,877474
3,SRF,283690
4,OMS,275731
5,IRP,213823
6,TRC,103175
7,MOT,81613
8,999,76527
9,OMR,52705


Passenger vehicls amount to 63%, Commercial Vehicles - 28.37% 

In [537]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
data_ny_pas = data_ny[data_ny['plate_type'] == 'PAS']
data_ny_pas.head()

,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00,2013-08-04 07:52:00,2013-08-04,2013-08-01,2013-08-01
6,GBH9379,PAS,NY,Sedan,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:37:00,2013-08-07 04:37:00,2013-08-07,2013-08-01,2013-08-01
8,M367CN,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 08:45:00,2013-07-18 08:45:00,2013-07-18,2013-07-18,2013-07-01
9,GAR6813,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 09:07:00,2013-07-18 09:07:00,2013-07-18,2013-07-18,2013-07-01
10,GAC2703,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,760,bwy,2013-08-12 18:56:00,2013-08-12 18:56:00,2013-08-12,2013-08-08,2013-08-01


In [538]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
data_ny_com = data_ny[data_ny['plate_type'] == 'COM']
data_ny_com.head()

,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00,2013-08-04 12:40:00,2013-08-04,2013-08-01,2013-08-01
2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00,2013-08-05 12:43:00,2013-08-05,2013-08-01,2013-08-01
3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00,2013-08-05 14:32:00,2013-08-05,2013-08-01,2013-08-01
11,40793JY,COM,NY,Van,14-No Standing,13,Manhattan,59,21st st,2013-07-05 23:45:00,2013-07-05 23:45:00,2013-07-05,2013-07-04,2013-07-01
21,84301JU,COM,NY,Pickup,14-No Standing,79,Brooklyn,737,myrtl ave,2013-07-19 08:50:00,2013-07-19 08:50:00,2013-07-19,2013-07-18,2013-07-01


In [539]:
#Which violations are the most popular for vehicles with passenger plates in NYC
nyc_violation_pas = data_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_violation_pas['percentage'] = (nyc_violation_pas['plate_id'] / 16451679)*100
nyc_violation_pas

,violation_description,plate_id,percentage
0,21-No Parking (street clean),3606164,21.919732
1,38-Failure to Display Muni Rec,2479152,15.069295
2,20A-No Parking (Non-COM),1766717,10.738825
3,37-Expired Muni Meter,1663027,10.108555
4,20-No Parking (Com Plate),1359018,8.260664
5,14-No Standing,1037409,6.305794
6,40-Fire Hydrant,959482,5.832122
7,70-Reg. Sticker Missing (NYS),810100,4.924117
8,46-Double Parking (Com Plate),658509,4.002686
9,19-No Stand (bus stop),391390,2.379028


In [540]:
fig = px.bar(nyc_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in NYC made by vehicles with passenger plates")
fig.show()

Most popular violation made by passenger cars in NYC is violation for no parking during street cleaning hours - 21.91% of the total number of violations by passenger cars. Following that is failure to display receipt - 15.06%, no parking violation for vehicles with non commercial plates - 10.74%, expired municipal meter - 10.10% and no parking for vehicles with commercial plate - 8.26%.

In [541]:
#Which violations are the most popular for vehicles with commercial plates in NYC
nyc_violation_com = data_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_violation_com['percentage'] = (nyc_violation_com['plate_id'] / 7406744)*100
nyc_violation_com

,violation_description,plate_id,percentage
0,14-No Standing,1137047,15.351509
1,38-Failure to Display Muni Rec,976347,13.181865
2,46-Double Parking (Com Plate),834402,11.265436
3,69-Failure to Disp Muni Recpt,707807,9.556250
4,37-Expired Muni Meter,443067,5.981940
5,16-No Std (Com Veh) Com Plate,358271,4.837092
6,21-No Parking (street clean),323041,4.361444
7,19-No Stand (bus stop),299907,4.049107
8,47-Double PKG-Midtown,275123,3.714493
9,20-No Parking (Com Plate),273364,3.690745


In [542]:
fig = px.bar(nyc_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in NYC made by vehicles with commercial plates")
fig.show()

Most popular commercial vehicle violations are no standing violations - 15.35%, failure to display municipal receipt - 13.18%, double parking - 11.26%, failure to display municipal receipt in a commercial zone - 9.56 and expired municipal meter - 5.98%.

In [543]:
#Which body types are the most popular out of all passenger vehicles in NYC
nyc_vehicle_type_pas = data_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_vehicle_type_pas['percentage'] = (nyc_vehicle_type_pas['plate_id'] / 16451679)*100
nyc_vehicle_type_pas

,vehicle_body_type,plate_id,percentage
0,Suburban,7922836,48.158221
1,Sedan,7825178,47.564616
2,Pickup,318884,1.938307
3,Van,204670,1.244068
4,Convertible,116890,0.710505
5,Others,43837,0.266459
6,Delivery Truck,8461,0.051429
7,Taxi,2805,0.017050
8,Bus,2718,0.016521
9,Utility,1561,0.009488


In [544]:
fig = px.bar(nyc_vehicle_type_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Passenger type vehicle body types that had the most violations in NYC")
fig.show()

Suburban and Sedan of all variations are the most popular vehicle body types of passenger vehicles in NYC - 48.16% and 47.56% respectively.

In [545]:
#Which body types are the most popular out of all commercial vehicles in NYC
nyc_vehicle_type_com = data_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_vehicle_type_com['percentage'] = (nyc_vehicle_type_com['plate_id'] /7406744) * 100
nyc_vehicle_type_com

,vehicle_body_type,plate_id,percentage
0,Van,4627193,62.472701
1,Delivery Truck,1520626,20.530290
2,Pickup,435413,5.878602
3,Utility,257621,3.478195
4,Refrigerator Trailer,228112,3.079788
5,Suburban,179721,2.426451
6,Others,65647,0.886314
7,Sedan,34731,0.468910
8,Flat Bed Truck,24985,0.337328
9,Tractor,11648,0.157262


In [546]:
fig = px.bar(nyc_vehicle_type_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in NYC")
fig.show()

The most popular vehicle body types for commercial vehicles are Vans, Delivery Trucks and Pickups - 62.47%, 20.53% and 5.88% respectively.

In [547]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
data_ny_pas_sub_sed = data_ny_pas[(data_ny_pas['vehicle_body_type'] == 'Sedan') | (data_ny_pas['vehicle_body_type'] == 'Suburban')]
data_ny_pas_sub_sed.head()

,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00,2013-08-04 07:52:00,2013-08-04,2013-08-01,2013-08-01
6,GBH9379,PAS,NY,Sedan,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:37:00,2013-08-07 04:37:00,2013-08-07,2013-08-01,2013-08-01
8,M367CN,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 08:45:00,2013-07-18 08:45:00,2013-07-18,2013-07-18,2013-07-01
9,GAR6813,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 09:07:00,2013-07-18 09:07:00,2013-07-18,2013-07-18,2013-07-01
10,GAC2703,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,760,bwy,2013-08-12 18:56:00,2013-08-12 18:56:00,2013-08-12,2013-08-08,2013-08-01


In [548]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
data_ny_pas_van_delv_pick = data_ny_com[(data_ny_com['vehicle_body_type'] == 'Van') | (data_ny_com['vehicle_body_type'] == 'Delivery Truck') | (data_ny_com['vehicle_body_type'] == 'Pickup')]
data_ny_pas_van_delv_pick.head()

,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00,2013-08-04 12:40:00,2013-08-04,2013-08-01,2013-08-01
2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00,2013-08-05 12:43:00,2013-08-05,2013-08-01,2013-08-01
3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00,2013-08-05 14:32:00,2013-08-05,2013-08-01,2013-08-01
11,40793JY,COM,NY,Van,14-No Standing,13,Manhattan,59,21st st,2013-07-05 23:45:00,2013-07-05 23:45:00,2013-07-05,2013-07-04,2013-07-01
21,84301JU,COM,NY,Pickup,14-No Standing,79,Brooklyn,737,myrtl ave,2013-07-19 08:50:00,2013-07-19 08:50:00,2013-07-19,2013-07-18,2013-07-01


In [549]:
#Which streets were the ones that had the most violations for passenger vehicles in NYC
nyc_name_street_pas = data_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_name_street_pas['percentage'] = (nyc_name_street_pas['plate_id'] / (7922836+7825178)) * 100
nyc_name_street_pas

,street_name,plate_id,percentage
0,broadway,315469,2.003230
1,3rd ave,198359,1.259581
2,5th ave,180170,1.144081
3,queens blvd,149740,0.950850
4,jamaica ave,118744,0.754025
5,coney island ave,103865,0.659543
6,37th ave,90136,0.572364
7,nostand ave,86528,0.549453
8,white plains rd,84035,0.533623
9,madison ave,83499,0.530219


In [550]:
fig = px.bar(nyc_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in NYC")
fig.show()

Streets with most number of violations by passenger vehicles in NYC are: Broadway - 2.00% of total number of violations, 3rd Avenue - 1.26%, 5th Avenue - 1.14%, Queens Boulevard - 0.95% and Jamaica Avenue - 0.75%

In [551]:
#Which streets were the ones that had the most violations for commercial vehicles in NYC
nyc_name_street_com = data_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_name_street_com['percentage'] = (nyc_name_street_com['plate_id'] / (4627193+1520626+435413)) * 100
nyc_name_street_com

,street_name,plate_id,percentage
0,broadway,303796,4.614694
1,3rd ave,242269,3.680092
2,madison ave,184802,2.807162
3,lexington ave,154179,2.341996
4,2nd ave,138108,2.097875
5,5th ave,114887,1.745146
6,1st ave,113652,1.726386
7,7th ave,91779,1.394133
8,8th ave,87229,1.325018
9,amsterdam ave,81847,1.243265


In [552]:
fig = px.bar(nyc_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for commercial vehicles in NYC")
fig.show()

Streets with most number of violations by commercial vehicles in NYC are: Broadway - 4.61% of total number of violations, 3rd Avenue - 3.68%, Madison Avenue - 2.81%, Lexington Avenue - 2.34% and 2nd Avenue - 2.10%

In [553]:
#Number of daily violations for passenger vehicles
data_daily_pas = data_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
data_daily_pas.head()

,day,plate_id
0,2013-07-01,32
1,2013-07-02,65
2,2013-07-03,53
3,2013-07-04,43
4,2013-07-05,52


In [554]:
fig = px.line(data_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall trand is stable, but there was a significant reduction in the number of violations from the end of May 2016 till the beginning of July 2016.

In [555]:
#Number of weekly violations for passenger vehicles
data_weekly_pas = data_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
data_weekly_pas.head()

,week,plate_id
0,2013-06-27,150
1,2013-07-04,661
2,2013-07-11,2685
3,2013-07-18,7939
4,2013-07-25,44773


In [556]:
fig = px.line(data_weekly_pas,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly trend of the total violations by passenger vehicles in NYC show that there was a stable amount of violations, usually with decreases in violations closer to the end of the year and with a drop in february of each year. However, there is a significant drop in the number of violations in May-June 2016.

In [557]:
#Number of monthly violations for passenger vehicles
data_monthly_pas = data_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
data_monthly_pas.head()

,month,plate_id
0,2013-07-01,56208
1,2013-08-01,349749
2,2013-09-01,349285
3,2013-10-01,415344
4,2013-11-01,386695


In [558]:
fig = px.line(data_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend for total number of passenger car violations show that there was pretty much same number of violations throughout these 4+ years. The only outlier is the decrease in violations in June 2016.

In [559]:
#Number of daily violations for commercial vehicles
data_daily_com = data_ny_com.groupby(['day'])['plate_id'].count().reset_index()
data_daily_com.head()

,day,plate_id
0,2013-07-01,41
1,2013-07-02,100
2,2013-07-03,132
3,2013-07-04,16
4,2013-07-05,97


In [560]:
fig = px.line(data_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars daily number of violations')
fig.show()

Daily violations in NYC made by commercial vehicle are very similar each year. There are much lower amount of violations on the weekends and at the end of each yeear. Apart from that, there is a slight decrease in daily violations in 2017, comparing to 2014-2016.

In [561]:
#Number of weekly violations for commercial vehicles
data_weekly_com = data_ny_com.groupby(['week'])['plate_id'].count().reset_index()
data_weekly_com.head()

,week,plate_id
0,2013-06-27,273
1,2013-07-04,992
2,2013-07-11,2431
3,2013-07-18,5336
4,2013-07-25,24556


In [562]:
fig = px.line(data_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars weekly number of violations')
fig.show()

Weekly trend of the number of violations made by commercial type of vehicles show that there is a clear decrease in the number of violations from the beginning of december until january of the following month. It is also clear that the number of weekly violations on average is lower in 2016 onwardsm comparing to 2013-2016

In [563]:
#Number of monthly violations for commercial vehicles
data_monthly_com = data_ny_com.groupby(['month'])['plate_id'].count().reset_index()
data_monthly_com.head()

,month,plate_id
0,2013-07-01,33588
1,2013-08-01,159133
2,2013-09-01,161117
3,2013-10-01,187209
4,2013-11-01,153324


In [564]:
fig = px.line(data_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars monthly number of violations')
fig.show()

Monthly trend of violations by commercial type of vehicles in NYC shows that usually there is a decrease in number of violations from october to december, december being the month with the lowest number of violations. Following that there is a steady increase in violations up until mrach and then the numbee of violations is staying the same. However, this trend began to change, as in March 2016, after peaking, number of violations decreased comparing to other years.

In [565]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in NYC
nyc_violations_precincts_pas = data_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_violations_precincts_pas['percentage'] = (nyc_violations_precincts_pas['plate_id'] / 16451679)*100
nyc_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,114,708523,4.306691
1,19,558429,3.394359
2,109,553759,3.365973
3,70,462984,2.814205
4,61,440712,2.678827
5,112,421814,2.563957
6,66,396365,2.409268
7,115,394125,2.395652
8,103,381576,2.319374
9,84,356065,2.164308


In [566]:
fig = px.bar(nyc_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             }, title="Precincts in NYC that gave the most number of tickets for passenger type vehicles")
fig.show()

Most popular/active precincts for passenger vehicles are 114th precinct - 4.31% of total violations, 19th precinct - 3.39%, 109th precinct - 3.37%, 70th precinct - 2.81% and 61st precinct - 2.68%

In [567]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in NYC
nyc_violations_precincts_com = data_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
nyc_violations_precincts_com['percentage'] = (nyc_violations_precincts_com['plate_id'] / 7406744)*100
nyc_violations_precincts_com

,violation_precinct,plate_id,percentage
0,19,970345,13.100831
1,14,657984,8.883580
2,18,620615,8.379053
3,1,575173,7.765531
4,13,420887,5.682483
5,17,364908,4.926699
6,20,286270,3.864991
7,10,210102,2.836631
8,84,186433,2.517071
9,6,186024,2.511549


In [568]:
fig = px.bar(nyc_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in NYC that gave the most number of tickets for commercial type vehicles")
fig.show()

Most popular/active precincts for commercial vehicles are 19th precinct - 13.1% of total violations, 14th precinct - 8.88%, 18th precinct - 8.38%, 1st precinct - 7.76% and 13th precinct - 5.68%

# PER COUNTY DATA EXPLORATION

# MANHATTAN

In [569]:
#Filter the main dataframe to contain only data related to the Manhattan county
manhattan = data[data['violation_county'] == 'Manhattan'].reset_index()
manhattan.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00,2013-08-04 07:52:00,2013-08-04,2013-08-01,2013-08-01
1,1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00,2013-08-04 12:40:00,2013-08-04,2013-08-01,2013-08-01
2,2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00,2013-08-05 12:43:00,2013-08-05,2013-08-01,2013-08-01
3,3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00,2013-08-05 14:32:00,2013-08-05,2013-08-01,2013-08-01
4,4,T60DAR,PAS,NJ,Pickup,14-No Standing,33,Manhattan,525,175th st,2013-08-11 18:17:00,2013-08-11 18:17:00,2013-08-11,2013-08-08,2013-08-01


In [570]:
#Which registration state is the most popular in Manhattan county
manhattan_reg_state = manhattan.groupby('registration_state')['plate_id'].count().reset_index()
manhattan_reg_state.head()

,registration_state,plate_id
0,99,30936
1,AB,441
2,AK,1028
3,AL,6573
4,AR,4210


In [571]:
fig = px.bar(manhattan_reg_state, x="registration_state", y="plate_id", labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             }, title="Number of violations in Manhattan by vehicle registration state")
fig.show()

71% of total number violations in Manhattan County is made by locals from NY. Another 15.55% are made by cars registered in NJ.

In [572]:
#Filter dataframe to contain only registration plates from NY
manhattan_ny = manhattan[manhattan['registration_state'] == 'NY']

In [573]:
#Which plate types are the most popular in Manhattan county
manhattan_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,COM,5372262
1,PAS,3631150
2,OMT,342775
3,IRP,162335
4,OMS,97385
5,SRF,75324
6,TRC,65343
7,MOT,52304
8,999,21199
9,MED,16039


Commercial vehicles ammount to 54.07% of the total violations in Manhattan made by NY locals. Passenger vehicles amount to another 36.55%. Omnibus taxis, which are also a commercial vehicle, amounts to another 3.45%

In [574]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
manhattan_ny_pas = manhattan_ny[manhattan_ny['plate_type'] == 'PAS']
manhattan_ny_pas.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00,2013-08-04 07:52:00,2013-08-04,2013-08-01,2013-08-01
6,18,GDE3973,PAS,NY,Sedan,40-Fire Hydrant,1,Manhattan,NaN,es river ter,2013-07-20 15:24:00,2013-07-20 15:24:00,2013-07-20,2013-07-18,2013-07-01
15,49,GHF1267,PAS,NY,Van,16-No Std (Com Veh) Com Plate,20,Manhattan,25,cpw,2013-07-21 14:48:00,2013-07-21 14:48:00,2013-07-21,2013-07-18,2013-07-01
16,50,578032,PAS,NY,Sedan,17-No Stand (exc auth veh),20,Manhattan,15,cpw,2013-07-30 12:03:00,2013-07-30 12:03:00,2013-07-30,2013-07-25,2013-07-01
18,52,GCX7943,PAS,NY,Sedan,17-No Stand (exc auth veh),20,Manhattan,15,cpw,2013-07-30 13:33:00,2013-07-30 13:33:00,2013-07-30,2013-07-25,2013-07-01


In [575]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
manhattan_ny_com = manhattan_ny[manhattan_ny['plate_type'] == 'COM']
manhattan_ny_com.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00,2013-08-04 12:40:00,2013-08-04,2013-08-01,2013-08-01
2,2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00,2013-08-05 12:43:00,2013-08-05,2013-08-01,2013-08-01
3,3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00,2013-08-05 14:32:00,2013-08-05,2013-08-01,2013-08-01
5,11,40793JY,COM,NY,Van,14-No Standing,13,Manhattan,59,21st st,2013-07-05 23:45:00,2013-07-05 23:45:00,2013-07-05,2013-07-04,2013-07-01
7,23,50097JX,COM,NY,Van,40-Fire Hydrant,5,Manhattan,124,hudson st,2013-08-07 10:23:00,2013-08-07 10:23:00,2013-08-07,2013-08-01,2013-08-01


In [576]:
#Which violations are the most popular for vehicles with passenger plates in Manhattan county
manhattan_violation_pas = manhattan_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_violation_pas['percentage'] = (manhattan_violation_pas['plate_id'] / 3631150)*100
manhattan_violation_pas

,violation_description,plate_id,percentage
0,21-No Parking (street clean),566354,15.597097
1,38-Failure to Display Muni Rec,466750,12.854055
2,37-Expired Muni Meter,410605,11.307850
3,14-No Standing,351159,9.670738
4,20-No Parking (Com Plate),326671,8.996351
5,20A-No Parking (Non-COM),275549,7.588477
6,31-No Stand (Com. Mtr. Zone),236155,6.503587
7,40-Fire Hydrant,233576,6.432563
8,16-No Std (Com Veh) Com Plate,147166,4.052876
9,70-Reg. Sticker Missing (NYS),132070,3.637140


In [577]:
fig = px.bar(manhattan_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Manhattan county made by vehicles with passenger plates")
fig.show()

For passenger vehicles, 15.60% of total tickets are for no parking during street cleaning hours, 12.85% are for failure to diplay receipt, 11.31% are for expired municipal meter, 9.67% are no standing violation and 9.00% are for no general parking violation for vehicles with commercial plates. 

In [578]:
#Which violations are the most popular for vehicles with commercial plates in Manhattan county
manhattan_violation_com = manhattan_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_violation_com['percentage'] = (manhattan_violation_com['plate_id'] / 5372262)*100
manhattan_violation_com

,violation_description,plate_id,percentage
0,14-No Standing,901221,16.775448
1,69-Failure to Disp Muni Recpt,703471,13.094503
2,46-Double Parking (Com Plate),610119,11.356836
3,38-Failure to Display Muni Rec,511584,9.522693
4,16-No Std (Com Veh) Com Plate,342561,6.376476
5,37-Expired Muni Meter,333986,6.216860
6,47-Double PKG-Midtown,275100,5.120748
7,42-Exp. Muni-Mtr (Com. Mtr. Z),263561,4.905960
8,20-No Parking (Com Plate),246263,4.583972
9,21-No Parking (street clean),186047,3.463104


In [579]:
fig = px.bar(manhattan_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Manhattan county made by vehicles with commercial plates")
fig.show()

For comercial vehicles, 16.77% of total tickers are for no standing violation, 13.09% are for failure to diplay receipt in the commercial zone, 11.36% are double parking, 9.52% are for failure to diplay receipt and 6.38% are for no standing for commercial vehicle violation.

In [580]:
#Which body types are the most popular out of all passenger vehicles in Manhattan county
manhattan_vehicle_type_pas = manhattan_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_vehicle_type_pas['percentage'] = (manhattan_vehicle_type_pas['plate_id'] / 3631150)*100
manhattan_vehicle_type_pas

,vehicle_body_type,plate_id,percentage
0,Suburban,1827836,50.337662
1,Sedan,1575477,43.387825
2,Pickup,106796,2.941107
3,Van,61301,1.688198
4,Convertible,40188,1.106757
5,Others,11744,0.323424
6,Delivery Truck,3878,0.106798
7,Taxi,1176,0.032386
8,Bus,656,0.018066
9,Motorcycle,610,0.016799


In [581]:
fig = px.bar(manhattan_vehicle_type_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Passenger type vehicle body types that had the most violations in Manhattan")
fig.show()

Regarding passenger type vehicle, vehicle bodytypes that received the highest ammount of tickets are Suburbans and Sedans - 50.34% and 43.39% respectively.

In [582]:
#Which body types are the most popular out of all commercial vehicles in Manhattan county
manhattan_vehicle_type_com = manhattan_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_vehicle_type_com['percentage'] = (manhattan_vehicle_type_com['plate_id'] / 5372262)*100
manhattan_vehicle_type_com

,vehicle_body_type,plate_id,percentage
0,Van,3519401,65.510599
1,Delivery Truck,981614,18.271894
2,Pickup,295470,5.499918
3,Refrigerator Trailer,178403,3.320817
4,Utility,160003,2.978317
5,Suburban,136716,2.544850
6,Others,39178,0.729265
7,Sedan,24996,0.465279
8,Flat Bed Truck,18265,0.339987
9,Tractor,6807,0.126706


In [583]:
fig = px.bar(manhattan_vehicle_type_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in Manhattan")
fig.show()

With regards to the body types of commercial vehicles, Vans, Delivery Trucks and Pickups received the most number of violations - 65.51%, 18.27% and 5.50% respectively.

In [584]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
manhattan_ny_pas_sub_sed = manhattan_ny_pas[(manhattan_ny_pas['vehicle_body_type'] == 'Sedan') | (manhattan_ny_pas['vehicle_body_type'] == 'Suburban')]
manhattan_ny_pas_sub_sed.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,0,GBB9093,PAS,NY,Suburban,46-Double Parking (Com Plate),33,Manhattan,712,175th st,2013-08-04 07:52:00,2013-08-04 07:52:00,2013-08-04,2013-08-01,2013-08-01
6,18,GDE3973,PAS,NY,Sedan,40-Fire Hydrant,1,Manhattan,NaN,es river ter,2013-07-20 15:24:00,2013-07-20 15:24:00,2013-07-20,2013-07-18,2013-07-01
16,50,578032,PAS,NY,Sedan,17-No Stand (exc auth veh),20,Manhattan,15,cpw,2013-07-30 12:03:00,2013-07-30 12:03:00,2013-07-30,2013-07-25,2013-07-01
18,52,GCX7943,PAS,NY,Sedan,17-No Stand (exc auth veh),20,Manhattan,15,cpw,2013-07-30 13:33:00,2013-07-30 13:33:00,2013-07-30,2013-07-25,2013-07-01
23,57,GDZ4899,PAS,NY,Suburban,17-No Stand (exc auth veh),20,Manhattan,15,cpw,2013-08-04 15:53:00,2013-08-04 15:53:00,2013-08-04,2013-08-01,2013-08-01


In [585]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
manhattan_ny_pas_van_delv_pick = manhattan_ny_com[(manhattan_ny_com['vehicle_body_type'] == 'Van') | (manhattan_ny_com['vehicle_body_type'] == 'Delivery Truck') | (manhattan_ny_com['vehicle_body_type'] == 'Pickup')]
manhattan_ny_pas_van_delv_pick.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,1,62416MB,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,201,177th st,2013-08-04 12:40:00,2013-08-04 12:40:00,2013-08-04,2013-08-01,2013-08-01
2,2,78755JZ,COM,NY,Pickup,46-Double Parking (Com Plate),33,Manhattan,520,163rd st,2013-08-05 12:43:00,2013-08-05 12:43:00,2013-08-05,2013-08-01,2013-08-01
3,3,63009MA,COM,NY,Van,46-Double Parking (Com Plate),33,Manhattan,517,176th st,2013-08-05 14:32:00,2013-08-05 14:32:00,2013-08-05,2013-08-01,2013-08-01
5,11,40793JY,COM,NY,Van,14-No Standing,13,Manhattan,59,21st st,2013-07-05 23:45:00,2013-07-05 23:45:00,2013-07-05,2013-07-04,2013-07-01
7,23,50097JX,COM,NY,Van,40-Fire Hydrant,5,Manhattan,124,hudson st,2013-08-07 10:23:00,2013-08-07 10:23:00,2013-08-07,2013-08-01,2013-08-01


In [586]:
#Which streets were the ones that had the most violations for passenger vehicles in Manhattan county
manhattan_name_street_pas = manhattan_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_name_street_pas['percentage'] = (manhattan_name_street_pas['plate_id'] / 3403313)*100
manhattan_name_street_pas

,street_name,plate_id,percentage
0,broadway,198719,5.838987
1,3rd ave,115260,3.386700
2,madison ave,83339,2.448761
3,amsterdam ave,71791,2.109445
4,1st ave,61449,1.805564
5,lexington ave,59752,1.755701
6,5th ave,54544,1.602674
7,2nd ave,54086,1.589216
8,columbus ave,44139,1.296942
9,st nicholas ave,36742,1.079595


In [587]:
fig = px.bar(manhattan_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in Manhattan")
fig.show()

The most number of violations in the Manhattan county that was made by passenger type vehicles are on Broadway - 5.83% of the total violations. Other streets are: 3rd Avenue - 3.39%, Madison Avenue - 2.45%, Amsterdam Avenue - 2.11% and 1st Avenue - 1.81%

In [588]:
#Which streets were the ones that had the most violations for commercial vehicles in Manhattan county
manhattan_name_street_com = manhattan_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_name_street_com['percentage'] = (manhattan_name_street_com['plate_id'] / 4796485)*100
manhattan_name_street_com

,street_name,plate_id,percentage
0,broadway,265164,5.528298
1,3rd ave,227059,4.733862
2,madison ave,184789,3.852592
3,lexington ave,153780,3.206098
4,2nd ave,137244,2.861345
5,1st ave,113183,2.359707
6,5th ave,88989,1.855296
7,8th ave,83427,1.739336
8,7th ave,83363,1.738002
9,amsterdam ave,81833,1.706104


In [589]:
fig = px.bar(manhattan_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for commercial vehicles in Manhattan")
fig.show()

Most popular streets for comercial vehicles violations are: Broadway - 5.53% of the total number violations and 3rd Avenue - 4.73%, Madison Avenue - 3.85%, Lexington Avenue - 3.21% and 2nd Avenue - 2.86%. 

3 out of 5 streets are the same streets that are most popular for passenger car violations as well.

In [590]:
#Number of daily violations for passenger vehicles
manhattan_daily_pas = manhattan_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
manhattan_daily_pas.head()

,day,plate_id
0,2013-07-01,16
1,2013-07-02,44
2,2013-07-03,28
3,2013-07-04,24
4,2013-07-05,37


In [591]:
fig = px.line(manhattan_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall trand is stable, but there was a number of significant reductions in the number of violations. The most apparent are the drop in violations between May and July 2016, apart from that, there are decreases in december, but it is maybe due to holidays.

In [592]:
#Number of weekly violations for passenger vehicles
manhattan_weekly_pas = manhattan_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
manhattan_weekly_pas.head()

,week,plate_id
0,2013-06-27,88
1,2013-07-04,469
2,2013-07-11,955
3,2013-07-18,1875
4,2013-07-25,10137


In [593]:
fig = px.line(manhattan_weekly_pas,
              x="week", 
              y="plate_id",labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly number of passenger type vehicle violations in Manhattan is pretty much consistent, apart from the decrease at the end of each year and subsequent rise to average level. The only significant drop which is difficult to explain is from March 2016 to middle of June 2016.

In [594]:
#Number of monthly violations for passenger vehicles
manhattan_monthly_pas = manhattan_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
manhattan_monthly_pas.head()

,month,plate_id
0,2013-07-01,13524
1,2013-08-01,83554
2,2013-09-01,89331
3,2013-10-01,99520
4,2013-11-01,92856


In [595]:
fig = px.line(manhattan_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend of the number of violations my passenger type vehicle in Manhattan show htat there is a clear drops in number of violations in each end of the year and they are pretty consistent, but there is also unexplained drop in March-June 2016.

In [596]:
#Number of daily violations for commercial vehicles
manhattan_daily_com = manhattan_ny_com.groupby(['day'])['plate_id'].count().reset_index()
manhattan_daily_com.head()

,day,plate_id
0,2013-07-01,39
1,2013-07-02,98
2,2013-07-03,129
3,2013-07-04,15
4,2013-07-05,96


In [597]:
fig = px.line(manhattan_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars daily number of violations')
fig.show()

Daily violations in Manhattan made by commercial vehicle are very similar each year. There are much lower amount of violations on the weekends and at the end of each yeear. Apart from that, there is a very slight decrease in daily violations in 2017, comparing to 2014-2016.

In [598]:
#Number of weekly violations for commercial vehicles
manhattan_weekly_com = manhattan_ny_com.groupby(['week'])['plate_id'].count().reset_index()
manhattan_weekly_com.head()

,week,plate_id
0,2013-06-27,266
1,2013-07-04,977
2,2013-07-11,1947
3,2013-07-18,3972
4,2013-07-25,18091


In [599]:
fig = px.line(manhattan_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars weekly number of violations')
fig.show()

Weekly trend of the number of violations made by commercial type of vehicles show that there is a clear decrease in the number of violations from the beginning of december until january of the following month. It is also clear that the number of weekly violations on average is lower in 2016 onwardsm comparing to 2013-2016

In [600]:
#Number of monthly violations for commercial vehicles
manhattan_monthly_com = manhattan_ny_com.groupby(['month'])['plate_id'].count().reset_index()
manhattan_monthly_com.head()

,month,plate_id
0,2013-07-01,25253
1,2013-08-01,115061
2,2013-09-01,115595
3,2013-10-01,134301
4,2013-11-01,110295


In [601]:
fig = px.line(manhattan_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars monthly number of violations')
fig.show()

Monthly trend of violations by commercial type of vehicles in NYC shows that usually there is a decrease in number of violations from october to december, december being the month with the lowest number of violations. Following that there is a steady increase in violations up until mrach and then the numbee of violations is staying the same. However, this trend began to change, as in March 2016, after peaking, number of violations decreased comparing to other years.

In [602]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in Manhattan county
manhattan_violations_precincts_pas = manhattan_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_violations_precincts_pas['percentage'] = (manhattan_violations_precincts_pas['plate_id'] / 3631150)*100
manhattan_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,19,558429,15.378847
1,13,232140,6.393016
2,20,228004,6.279113
3,1,217563,5.991573
4,6,189779,5.226416
5,34,189537,5.219751
6,18,180203,4.962698
7,24,172948,4.762899
8,9,167933,4.624788
9,5,165760,4.564945


In [603]:
fig = px.bar(manhattan_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Manhattan that gave the most number of tickets for passenger type vehicles")
fig.show()

Precinct that gave by far the biggest number of tickets to the passeger type vehicle is 19th precinct - 15.37% of total tickets.

In [604]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in Manhattan county
manhattan_violations_precincts_com = manhattan_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
manhattan_violations_precincts_com['percentage'] = (manhattan_violations_precincts_com['plate_id'] / 5372262)*100
manhattan_violations_precincts_com

,violation_precinct,plate_id,percentage
0,19,970345,18.062131
1,14,657984,12.247802
2,18,620615,11.552210
3,1,575173,10.706347
4,13,420887,7.834447
5,17,364908,6.792446
6,20,286270,5.328668
7,10,210102,3.910867
8,6,186024,3.462675
9,5,179615,3.343378


In [605]:
fig = px.bar(manhattan_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Manhattan that gave the most number of tickets for commercial type vehicles")
fig.show()

There are 4 precincts that gave the most number of tickets to the commercial type vehicles: 19th precints - 18.06% of total tickets, 14th precinct - 12.25%, 18th precinct - 11.56% and 1st precinct - 10.71%

# BROOKLYN

In [606]:
#Filter the main dataframe to contain only data related to the Brooklyn county
brooklyn = data[data['violation_county'] == 'Brooklyn'].reset_index()
brooklyn.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,5,XZ764G,PAS,NJ,Delivery Truck,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:25:00,2013-08-07 04:25:00,2013-08-07,2013-08-01,2013-08-01
1,6,GBH9379,PAS,NY,Sedan,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:37:00,2013-08-07 04:37:00,2013-08-07,2013-08-01,2013-08-01
2,7,MCL78B,PAS,NJ,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookl,2013-07-18 08:39:00,2013-07-18 08:39:00,2013-07-18,2013-07-18,2013-07-01
3,8,M367CN,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 08:45:00,2013-07-18 08:45:00,2013-07-18,2013-07-18,2013-07-01
4,9,GAR6813,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 09:07:00,2013-07-18 09:07:00,2013-07-18,2013-07-18,2013-07-01


In [607]:
#Which registration state is the most popular in Brooklyn county
brooklyn_reg_state = brooklyn.groupby('registration_state')['plate_id'].count().reset_index()
brooklyn_reg_state.head()

,registration_state,plate_id
0,99,27384
1,AB,352
2,AK,1269
3,AL,10453
4,AR,3187


In [608]:
fig = px.bar(brooklyn_reg_state, x="registration_state", y="plate_id",labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             }, title="Number of violations in Brooklyn by vehicle registration state")
fig.show()

78.46% of total number violations in Brooklyn County is made by locals from NY. 

In [609]:
#Filter dataframe to contain only registration plates from NY
brooklyn_ny = brooklyn[brooklyn['registration_state'] == 'NY']

In [610]:
#Which plate types are the most popular in Brooklyn county
brooklyn_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,PAS,5300785
1,COM,724341
2,OMT,174683
3,SRF,86339
4,OMS,86033
5,999,27482
6,OMR,23371
7,MOT,15967
8,ORG,12978
9,IRP,11635


Passener vehicles ammount to 81.08% of the total violations in Brooklyn made by NY locals. Commercial vehicles amount to another 11.08%. Omnibus taxis, which are also a commercial vehicle, amounts to another 2.67%

In [611]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
brooklyn_ny_pas = brooklyn_ny[brooklyn_ny['plate_type'] == 'PAS']
brooklyn_ny_pas.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,6,GBH9379,PAS,NY,Sedan,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:37:00,2013-08-07 04:37:00,2013-08-07,2013-08-01,2013-08-01
3,8,M367CN,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 08:45:00,2013-07-18 08:45:00,2013-07-18,2013-07-18,2013-07-01
4,9,GAR6813,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 09:07:00,2013-07-18 09:07:00,2013-07-18,2013-07-18,2013-07-01
5,10,GAC2703,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,760,bwy,2013-08-12 18:56:00,2013-08-12 18:56:00,2013-08-12,2013-08-08,2013-08-01
6,12,GAD1485,PAS,NY,Sedan,20-No Parking (Com Plate),76,Brooklyn,221,congress st,2013-08-12 17:46:00,2013-08-12 17:46:00,2013-08-12,2013-08-08,2013-08-01


In [612]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
brooklyn_ny_com = brooklyn_ny[brooklyn_ny['plate_type'] == 'COM']
brooklyn_ny_com.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
14,21,84301JU,COM,NY,Pickup,14-No Standing,79,Brooklyn,737,myrtl ave,2013-07-19 08:50:00,2013-07-19 08:50:00,2013-07-19,2013-07-18,2013-07-01
118,237,54019JX,COM,NY,Van,21-No Parking (street clean),71,Brooklyn,NaN,ss lefferts ave,2013-07-25 11:35:00,2013-07-25 11:35:00,2013-07-25,2013-07-25,2013-07-01
162,480,51632AP,COM,NY,Van,21-No Parking (street clean),66,Brooklyn,5702,17th ave,2013-07-23 09:58:00,2013-07-23 09:58:00,2013-07-23,2013-07-18,2013-07-01
255,603,86737MC,COM,NY,Pickup,21-No Parking (street clean),71,Brooklyn,NaN,ss of eastern parkwy,2013-08-06 11:18:00,2013-08-06 11:18:00,2013-08-06,2013-08-01,2013-08-01
272,666,54260MB,COM,NY,Utility,21-No Parking (street clean),61,Brooklyn,1722,kings hwy,2013-08-22 08:12:00,2013-08-22 08:12:00,2013-08-22,2013-08-22,2013-08-01


In [613]:
#Which violations are the most popular for vehicles with passenger plates in Brooklyn county
brooklyn_violation_pas = brooklyn_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_violation_pas['percentage'] = (brooklyn_violation_pas['plate_id'] / 5300785)*100
brooklyn_violation_pas

,violation_description,plate_id,percentage
0,21-No Parking (street clean),1536509,28.986443
1,38-Failure to Display Muni Rec,825947,15.581598
2,20A-No Parking (Non-COM),502267,9.475332
3,37-Expired Muni Meter,465473,8.781209
4,20-No Parking (Com Plate),403420,7.610571
5,40-Fire Hydrant,312981,5.904427
6,14-No Standing,293896,5.544386
7,46-Double Parking (Com Plate),246922,4.658216
8,70-Reg. Sticker Missing (NYS),216110,4.076943
9,19-No Stand (bus stop),131178,2.474690


In [614]:
fig = px.bar(brooklyn_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Brooklyn county made by vehicles with passenger plates")
fig.show()

For passenger vehicles, 28.99% are for no parking during street cleaning hours, 15.58% are for failure to diplay receipt, 9.47% are no general parking for non commercial vehicle, 8.78% for expired municipal meter and 7.61% for no general parking for vehicles with commercial plates.

In [615]:
#Which violations are the most popular for vehicles with commercial plates in Brooklyn county
brooklyn_violation_com = brooklyn_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_violation_com['percentage'] = (brooklyn_violation_com['plate_id'] / 724341)*100
brooklyn_violation_com

,violation_description,plate_id,percentage
0,38-Failure to Display Muni Rec,130839,18.063177
1,14-No Standing,104876,14.478816
2,46-Double Parking (Com Plate),91167,12.586199
3,19-No Stand (bus stop),69328,9.571183
4,40-Fire Hydrant,63723,8.797376
5,78-Nighttime PKG on Res Street,49660,6.855887
6,21-No Parking (street clean),45807,6.323955
7,37-Expired Muni Meter,35390,4.885820
8,20A-No Parking (Non-COM),20418,2.818838
9,48-Bike Lane,19704,2.720266


In [616]:
fig = px.bar(brooklyn_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Brooklyn county made by vehicles with commercial plates")
fig.show()

For comercial vehicles, 18.06% are for for failure to diplay receipt, 14.46% are for no standing violation, 12.59% are for double parking, 9.57% are for no standing at the bus stop violation and 8.80% are for standing next to fire hydrant.

In [617]:
#Which body types are the most popular out of all passenger vehicles in Brooklyn county
brooklyn_vehicle_type_pas = brooklyn_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_vehicle_type_pas['percentage'] = (brooklyn_vehicle_type_pas['plate_id'] / 5300785)*100
brooklyn_vehicle_type_pas

,vehicle_body_type,plate_id,percentage
0,Suburban,2643531,49.870557
1,Sedan,2463202,46.468627
2,Pickup,87779,1.655962
3,Van,56065,1.057674
4,Convertible,31772,0.599383
5,Others,13119,0.247492
6,Delivery Truck,2518,0.047502
7,Bus,888,0.016752
8,Taxi,524,0.009885
9,Utility,368,0.006942


In [618]:
fig = px.bar(brooklyn_vehicle_type_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Passenger type vehicle body types that had the most violations in Brooklyn")
fig.show()

As in other counties, Suburbans and Sedans of various types are the body types for passenger vehicles that received the most number of violations - 49.87% and 46.47% respectively.

In [619]:
#Which body types are the most popular out of all commercial vehicles in Brooklyn county
brooklyn_vehicle_type_com = brooklyn_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_vehicle_type_com['percentage'] = (brooklyn_vehicle_type_com['plate_id'] / 724341)*100
brooklyn_vehicle_type_com

,vehicle_body_type,plate_id,percentage
0,Van,387962,53.560685
1,Delivery Truck,213159,29.427990
2,Pickup,43172,5.960176
3,Utility,27481,3.793931
4,Suburban,18372,2.536374
5,Refrigerator Trailer,14304,1.974761
6,Others,9607,1.326309
7,Sedan,3520,0.485959
8,Flat Bed Truck,2490,0.343761
9,Tractor,1292,0.178369


In [620]:
fig = px.bar(brooklyn_vehicle_type_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in Brooklyn")
fig.show()

With regards to the body types of commercial vehicles, Vans, Delivery Trucks and Pickups received the most number of violations - 53.56%, 29.43% and 5.96% respectively.

In [621]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
brooklyn_ny_pas_sub_sed = brooklyn_ny_pas[(brooklyn_ny_pas['vehicle_body_type'] == 'Sedan') | (brooklyn_ny_pas['vehicle_body_type'] == 'Suburban')]
brooklyn_ny_pas_sub_sed.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,6,GBH9379,PAS,NY,Sedan,24-No Parking (exc auth veh),88,Brooklyn,100,nportland ave,2013-08-07 04:37:00,2013-08-07 04:37:00,2013-08-07,2013-08-01,2013-08-01
3,8,M367CN,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 08:45:00,2013-07-18 08:45:00,2013-07-18,2013-07-18,2013-07-01
4,9,GAR6813,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,NaN,760th broadway brookly,2013-07-18 09:07:00,2013-07-18 09:07:00,2013-07-18,2013-07-18,2013-07-01
5,10,GAC2703,PAS,NY,Sedan,24-No Parking (exc auth veh),79,Brooklyn,760,bwy,2013-08-12 18:56:00,2013-08-12 18:56:00,2013-08-12,2013-08-08,2013-08-01
6,12,GAD1485,PAS,NY,Sedan,20-No Parking (Com Plate),76,Brooklyn,221,congress st,2013-08-12 17:46:00,2013-08-12 17:46:00,2013-08-12,2013-08-08,2013-08-01


In [622]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
brooklyn_ny_pas_van_delv_pick = brooklyn_ny_com[(brooklyn_ny_com['vehicle_body_type'] == 'Van') | (brooklyn_ny_com['vehicle_body_type'] == 'Delivery Truck') | (brooklyn_ny_com['vehicle_body_type'] == 'Pickup')]
brooklyn_ny_pas_van_delv_pick.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
14,21,84301JU,COM,NY,Pickup,14-No Standing,79,Brooklyn,737,myrtl ave,2013-07-19 08:50:00,2013-07-19 08:50:00,2013-07-19,2013-07-18,2013-07-01
118,237,54019JX,COM,NY,Van,21-No Parking (street clean),71,Brooklyn,NaN,ss lefferts ave,2013-07-25 11:35:00,2013-07-25 11:35:00,2013-07-25,2013-07-25,2013-07-01
162,480,51632AP,COM,NY,Van,21-No Parking (street clean),66,Brooklyn,5702,17th ave,2013-07-23 09:58:00,2013-07-23 09:58:00,2013-07-23,2013-07-18,2013-07-01
255,603,86737MC,COM,NY,Pickup,21-No Parking (street clean),71,Brooklyn,NaN,ss of eastern parkwy,2013-08-06 11:18:00,2013-08-06 11:18:00,2013-08-06,2013-08-01,2013-08-01
275,717,54387MB,COM,NY,Pickup,21-No Parking (street clean),70,Brooklyn,1419,foster ave,2013-07-22 08:30:00,2013-07-22 08:30:00,2013-07-22,2013-07-18,2013-07-01


In [623]:
#Which streets were the ones that had the most violations for passenger vehicles in Brooklyn county
brooklyn_name_street_pas = brooklyn_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_name_street_pas['percentage'] = (brooklyn_name_street_pas['plate_id'] / 5106733)*100
brooklyn_name_street_pas

,street_name,plate_id,percentage
0,5th ave,125346,2.454524
1,coney island ave,103862,2.033825
2,nostand ave,86493,1.693705
3,flatbush ave,70695,1.384349
4,13th ave,68373,1.338879
5,3rd ave,63652,1.246433
6,7th ave,58599,1.147485
7,atlantic ave,56368,1.103798
8,86th st,50078,0.980627
9,fulton st,49622,0.971698


In [624]:
fig = px.bar(brooklyn_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in Brooklyn")
fig.show()

The most number of violations in the Brooklyn county that has made passenger cars are on 5th Avenue - 2.45% of the total violations. Other streets are: Coney Island Avenue - 2.03%, Nostrand Avenue - 1.69%, Flatbush Avenue - 1.38% and 13th Avenue - 1.34%.

In [625]:
#Which streets were the ones that had the most violations for commercial vehicles in Brooklyn county
brooklyn_name_street_com = brooklyn_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_name_street_com['percentage'] = (brooklyn_name_street_com['plate_id'] / 644293)*100
brooklyn_name_street_com

,street_name,plate_id,percentage
0,court st,28900,4.485537
1,5th ave,25849,4.011995
2,fulton st,14869,2.307801
3,livingston st,13048,2.025166
4,atlantic ave,12801,1.986829
5,flatbush ave,12180,1.890444
6,manhattan ave,12157,1.886874
7,3rd ave,9973,1.547898
8,montague st,9772,1.516701
9,nostand ave,9435,1.464396


In [626]:
fig = px.bar(brooklyn_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for commercial vehicles in Brooklyn")
fig.show()

Most popular streets for comercial vehicles violations are: Court Street - 4.48% of the total number violations and 5th Avenue - 4.01%. Other streets are: Fulton Street - 2.31%, Livingston Street - 2.02% and Atlantic Avenue - 1.99%. 

In [627]:
#Number of daily violations for passenger vehicles
brooklyn_daily_pas = brooklyn_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
brooklyn_daily_pas.head()

,day,plate_id
0,2013-07-01,4
1,2013-07-02,12
2,2013-07-03,12
3,2013-07-04,11
4,2013-07-05,5


In [628]:
fig = px.line(brooklyn_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall trand is stable, but there was a significant drop in violations between May and July 2016.

In [629]:
#Number of weekly violations for passenger vehicles
brooklyn_weekly_pas = brooklyn_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
brooklyn_weekly_pas.head()

,week,plate_id
0,2013-06-27,28
1,2013-07-04,121
2,2013-07-11,1288
3,2013-07-18,3097
4,2013-07-25,14451


In [630]:
fig = px.line(brooklyn_weekly_pas,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly number of passenger type vehicle violations in Brooklyn is pretty much consistent, apart from the decrease at the end of each year and subsequent rise to average level. The only significant drop which is difficult to explain is from March 2016 to middle of June 2016.

In [631]:
#Number of monthly violations for passenger vehicles
brooklyn_monthly_pas = brooklyn_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
brooklyn_monthly_pas.head()

,month,plate_id
0,2013-07-01,18985
1,2013-08-01,108544
2,2013-09-01,98438
3,2013-10-01,132709
4,2013-11-01,120125


In [632]:
fig = px.line(brooklyn_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend shows that there is a overall similar amount of violations made py passenger type vehicles. There is a surprising drop in May-June 2016. 

In [633]:
#Number of daily violations for commercial vehicles
brooklyn_daily_com = brooklyn_ny_com.groupby(['day'])['plate_id'].count().reset_index()
brooklyn_daily_com.head()

,day,plate_id
0,2013-07-02,2
1,2013-07-03,1
2,2013-07-04,1
3,2013-07-05,1
4,2013-07-06,1


In [634]:
fig = px.line(brooklyn_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars daily number of violations')
fig.show()

Daily trend of violations made by commercial type of vehicles in Brooklyn show similar picture as in other counties - number of violations by commercial type vehicles decrease on the weekends and at the end of each year. Overall the number of violations from 2016 is lower than it was before.

In [635]:
#Number of weekly violations for commercial vehicles
brooklyn_weekly_com = brooklyn_ny_com.groupby(['week'])['plate_id'].count().reset_index()
brooklyn_weekly_com.head()

,week,plate_id
0,2013-06-27,3
1,2013-07-04,9
2,2013-07-11,399
3,2013-07-18,814
4,2013-07-25,2567


In [636]:
fig = px.line(brooklyn_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars weekly number of violations')
fig.show()

Weekly trend shows that there is a decrease at the end of each year and that there is a lower on average number of violations in 2016 onwards comparing to previous years. 

In [637]:
#Number of monthly violations for commercial vehicles
brooklyn_monthly_com = brooklyn_ny_com.groupby(['month'])['plate_id'].count().reset_index()
brooklyn_monthly_com.head()

,month,plate_id
0,2013-07-01,3792
1,2013-08-01,16935
2,2013-09-01,17409
3,2013-10-01,20427
4,2013-11-01,16239


In [638]:
fig = px.line(brooklyn_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars monthly number of violations')
fig.show()

Monthly trend of violations made by commercial type of vehicles in Brooklyn shows that at the end of each year there is the lowest amount of violations and that the number of monthly violations in 2016 onwards has decreased, comparing to previous years. 

In [639]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in Brooklyn county
brooklyn_violations_precincts_pas = brooklyn_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(15)
brooklyn_violations_precincts_pas['percentage'] = (brooklyn_violations_precincts_pas['plate_id'] / 5300785)*100
brooklyn_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,70,462984,8.734254
1,61,440712,8.314089
2,66,396365,7.477477
3,84,356065,6.717213
4,62,338356,6.383130
5,68,332836,6.278995
6,90,323153,6.096323
7,78,289508,5.461606
8,94,222631,4.199963
9,67,221380,4.176363


In [640]:
fig = px.bar(brooklyn_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Brooklyn that gave the most number of tickets for passenger type vehicles")
fig.show()

Precincts that gave the most number of tickets in Brooklyn for passenger type vehicles are: 70th precinct - 8.73% of total tickets to passenger type vehicles, 61st precinct - 8.31%, 66th precinct - 7.48%

In [641]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in Queens county
brooklyn_violations_precincts_com = brooklyn_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
brooklyn_violations_precincts_com['percentage'] = (brooklyn_violations_precincts_com['plate_id'] / 724341)*100
brooklyn_violations_precincts_com

,violation_precinct,plate_id,percentage
0,84,186433,25.738292
1,90,51259,7.076639
2,68,46272,6.388151
3,94,43052,5.943609
4,88,42296,5.839239
5,78,40800,5.632706
6,70,36916,5.096495
7,61,33953,4.687433
8,66,32661,4.509064
9,62,29739,4.105663


In [642]:
fig = px.bar(brooklyn_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Brooklyn that gave the most number of tickets for commercial type vehicles")
fig.show()

Precinct that gave by far the most number of tickets to commercial type vehicles is 84th precinct - 25.74% of total tickets to commercial type vehicles.

# QUEENS

In [643]:
#Filter the main dataframe to contain only data related to the Queens county
queens = data[data['violation_county'] == 'Queens'].reset_index()
queens.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,30,GBC9079,PAS,NY,Sedan,98-Obstructing Driveway,107,Queens,170-10,69th ave,2013-07-27 09:27:00,2013-07-27 09:27:00,2013-07-27,2013-07-25,2013-07-01
1,31,42587JW,COM,NY,Delivery Truck,46-Double Parking (Com Plate),107,Queens,189-17,union tpark,2013-07-30 11:19:00,2013-07-30 11:19:00,2013-07-30,2013-07-25,2013-07-01
2,32,SES174,PAS,NB,Sedan,19-No Stand (bus stop),107,Queens,179-09,hillsid ave,2013-08-15 10:50:00,2013-08-15 10:50:00,2013-08-15,2013-08-15,2013-08-01
3,37,GBB7685,PAS,NY,Sedan,19-No Stand (bus stop),112,Queens,118-20,queens blvd,2013-07-14 18:00:00,2013-07-14 18:00:00,2013-07-14,2013-07-11,2013-07-01
4,38,904YDS,PAS,CT,Sedan,19-No Stand (bus stop),109,Queens,36-26,main st,2013-07-14 18:10:00,2013-07-14 18:10:00,2013-07-14,2013-07-11,2013-07-01


In [644]:
#Which registration state is the most popular in Queens county
queens_reg_state = queens.groupby('registration_state')['plate_id'].count().reset_index()
queens_reg_state.head()

,registration_state,plate_id
0,99,17059
1,AB,164
2,AK,710
3,AL,3987
4,AR,2059


In [645]:
fig = px.bar(queens_reg_state, x="registration_state", y="plate_id", labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             }, title="Number of violations in Queens by vehicle registration state")
fig.show()

84.75% of total number violations in Queens County is made by locals from NY. 

In [646]:
#Filter dataframe to contain only registration plates from NY
queens_ny = queens[queens['registration_state'] == 'NY']

In [647]:
#Which plate types are the most popular in Queens county
queens_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,PAS,4715404
1,COM,914308
2,OMT,217042
3,SRF,81950
4,OMS,58272
5,IRP,30815
6,TRC,22310
7,999,16350
8,ORG,12937
9,OMR,11627


Passener vehicles ammount to 76.7% of the total violations in Queens made by NY locals. Commercial vehicles amount to another 14.87%. Omnibus taxis, which are also a commercial vehicle, amounts to another 3.53%

In [648]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
queens_ny_pas = queens_ny[queens_ny['plate_type'] == 'PAS']
queens_ny_pas.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,30,GBC9079,PAS,NY,Sedan,98-Obstructing Driveway,107,Queens,170-10,69th ave,2013-07-27 09:27:00,2013-07-27 09:27:00,2013-07-27,2013-07-25,2013-07-01
3,37,GBB7685,PAS,NY,Sedan,19-No Stand (bus stop),112,Queens,118-20,queens blvd,2013-07-14 18:00:00,2013-07-14 18:00:00,2013-07-14,2013-07-11,2013-07-01
9,99,GHM9264,PAS,NY,Suburban,63-Nighttime STD/PKG in a Park,100,Queens,NaN,riis park main lot,2013-07-20 00:45:00,2013-07-20 00:45:00,2013-07-20,2013-07-18,2013-07-01
12,104,54RD87,PAS,NY,Others,20-No Parking (Com Plate),100,Queens,NaN,beach 169th st,2013-07-15 17:01:00,2013-07-15 17:01:00,2013-07-15,2013-07-11,2013-07-01
13,105,GFP5637,PAS,NY,Suburban,20-No Parking (Com Plate),100,Queens,NaN,albanners east,2013-07-18 23:25:00,2013-07-18 23:25:00,2013-07-18,2013-07-18,2013-07-01


In [649]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
queens_ny_com = queens_ny[queens_ny['plate_type'] == 'COM']
queens_ny_com.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,31,42587JW,COM,NY,Delivery Truck,46-Double Parking (Com Plate),107,Queens,189-17,union tpark,2013-07-30 11:19:00,2013-07-30 11:19:00,2013-07-30,2013-07-25,2013-07-01
8,79,42909JM,COM,NY,Delivery Truck,19-No Stand (bus stop),103,Queens,88-28,parsons blvd,2013-07-17 16:30:00,2013-07-17 16:30:00,2013-07-17,2013-07-11,2013-07-01
15,110,97732JZ,COM,NY,Refrigerator Trailer,46-Double Parking (Com Plate),108,Queens,21-10,borden ave,2013-08-15 23:50:00,2013-08-15 23:50:00,2013-08-15,2013-08-15,2013-08-01
16,111,91628MC,COM,NY,Refrigerator Trailer,46-Double Parking (Com Plate),108,Queens,21-51,borden ave,2013-08-02 04:48:00,2013-08-02 04:48:00,2013-08-02,2013-08-01,2013-08-01
17,112,84614JR,COM,NY,Others,46-Double Parking (Com Plate),108,Queens,21-09,borden ave,2013-08-23 04:42:00,2013-08-23 04:42:00,2013-08-23,2013-08-22,2013-08-01


In [650]:
#Which violations are the most popular for vehicles with passenger plates in Queens county
queens_violation_pas = queens_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_violation_pas['percentage'] = (queens_violation_pas['plate_id'] / 4715404)*100
queens_violation_pas

,violation_description,plate_id,percentage
0,21-No Parking (street clean),924221,19.600039
1,38-Failure to Display Muni Rec,813948,17.261469
2,20A-No Parking (Non-COM),609412,12.923856
3,37-Expired Muni Meter,570655,12.101932
4,20-No Parking (Com Plate),391047,8.292969
5,70-Reg. Sticker Missing (NYS),303236,6.430753
6,14-No Standing,254699,5.401425
7,40-Fire Hydrant,234995,4.983560
8,19-No Stand (bus stop),103369,2.192156
9,46-Double Parking (Com Plate),92246,1.956269


In [651]:
fig = px.bar(queens_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Queens county made by vehicles with passenger plates")
fig.show()

For passenger vehicles, 19.60% are for no parking during street cleaning hours, 17.26% are for failure to diplay receipt, 12.92% are no general parking for non commercial vehicle, 12.10% for expired municipal meter and 8.29% are for no general parking for vehicles with commercial plate.

In [652]:
#Which violations are the most popular for vehicles with commercial plates in Queens county
queens_violation_com = queens_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_violation_com['percentage'] = (queens_violation_com['plate_id'] / 914308)*100
queens_violation_com

,violation_description,plate_id,percentage
0,38-Failure to Display Muni Rec,259486,28.380589
1,14-No Standing,102832,11.246976
2,19-No Stand (bus stop),82347,9.006484
3,40-Fire Hydrant,65304,7.142451
4,21-No Parking (street clean),60902,6.660994
5,46-Double Parking (Com Plate),58169,6.362079
6,37-Expired Muni Meter,54492,5.959917
7,78-Nighttime PKG on Res Street,53180,5.816421
8,85-Storage-3 hour Commercial,36637,4.007074
9,20A-No Parking (Non-COM),26166,2.861836


In [653]:
fig = px.bar(queens_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Queens county made by vehicles with commercial plates")
fig.show()

For comercial vehicles, 28.38% are for for failure to diplay receipt, 11.24% are for no standing violation, 9.00% are for no standing at the bus stop violation, 7.14% are for standing next to fire hydrant and 6.66% are for parking during street cleaning hours.=.

In [654]:
#Which body types are the most popular out of all passenger vehicles in Queens county
queens_vehicle_type_pas = queens_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_vehicle_type_pas['percentage'] = (queens_vehicle_type_pas['plate_id'] / 4715404)*100
queens_vehicle_type_pas

,vehicle_body_type,plate_id,percentage
0,Sedan,2382128,50.518004
1,Suburban,2162944,45.869749
2,Pickup,80874,1.715102
3,Van,44134,0.935954
4,Convertible,31623,0.670632
5,Others,9633,0.204288
6,Delivery Truck,1353,0.028693
7,Bus,850,0.018026
8,Taxi,648,0.013742
9,Motorcycle,344,0.007295


In [655]:
fig = px.bar(queens_vehicle_type_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Passenger type vehicle body types that had the most violations in Queens")
fig.show()

As in Bronx, Sedans of various types and Suburbans are the body types for vehicles that received the most number of violations - 50.52% and 45.87% respectively.

In [656]:
#Which body types are the most popular out of all commercial vehicles in Queens county
queens_vehicle_type_com = queens_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_vehicle_type_com['percentage'] = (queens_vehicle_type_com['plate_id'] / 914308)*100
queens_vehicle_type_com

,vehicle_body_type,plate_id,percentage
0,Van,458875,50.188230
1,Delivery Truck,254640,27.850571
2,Pickup,74858,8.187394
3,Utility,57162,6.251941
4,Refrigerator Trailer,27874,3.048644
5,Suburban,15611,1.707412
6,Others,11991,1.311484
7,Sedan,3965,0.433661
8,Flat Bed Truck,3258,0.356335
9,Tractor,2082,0.227713


In [657]:
fig = px.bar(queens_vehicle_type_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in Queens")
fig.show()

With regards to the body types of commercial vehicles, Vans, Delivery Trucks and Pickups received the most number of violations - 50.19%, 27.85% and 8.19% respectively.

In [658]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
queens_ny_pas_sub_sed = queens_ny_pas[(queens_ny_pas['vehicle_body_type'] == 'Sedan') | (queens_ny_pas['vehicle_body_type'] == 'Suburban')]
queens_ny_pas_sub_sed.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,30,GBC9079,PAS,NY,Sedan,98-Obstructing Driveway,107,Queens,170-10,69th ave,2013-07-27 09:27:00,2013-07-27 09:27:00,2013-07-27,2013-07-25,2013-07-01
3,37,GBB7685,PAS,NY,Sedan,19-No Stand (bus stop),112,Queens,118-20,queens blvd,2013-07-14 18:00:00,2013-07-14 18:00:00,2013-07-14,2013-07-11,2013-07-01
9,99,GHM9264,PAS,NY,Suburban,63-Nighttime STD/PKG in a Park,100,Queens,NaN,riis park main lot,2013-07-20 00:45:00,2013-07-20 00:45:00,2013-07-20,2013-07-18,2013-07-01
13,105,GFP5637,PAS,NY,Suburban,20-No Parking (Com Plate),100,Queens,NaN,albanners east,2013-07-18 23:25:00,2013-07-18 23:25:00,2013-07-18,2013-07-18,2013-07-01
14,106,GGH6934,PAS,NY,Sedan,20-No Parking (Com Plate),100,Queens,NaN,n channel bridge,2013-07-27 23:36:00,2013-07-27 23:36:00,2013-07-27,2013-07-25,2013-07-01


In [659]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
queens_ny_pas_van_delv_pick = queens_ny_com[(queens_ny_com['vehicle_body_type'] == 'Van') | (queens_ny_com['vehicle_body_type'] == 'Delivery Truck') | (queens_ny_com['vehicle_body_type'] == 'Pickup')]
queens_ny_pas_van_delv_pick.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
1,31,42587JW,COM,NY,Delivery Truck,46-Double Parking (Com Plate),107,Queens,189-17,union tpark,2013-07-30 11:19:00,2013-07-30 11:19:00,2013-07-30,2013-07-25,2013-07-01
8,79,42909JM,COM,NY,Delivery Truck,19-No Stand (bus stop),103,Queens,88-28,parsons blvd,2013-07-17 16:30:00,2013-07-17 16:30:00,2013-07-17,2013-07-11,2013-07-01
49,351,80032KA,COM,NY,Delivery Truck,20-No Parking (Com Plate),103,Queens,88-24,merrick blvd,2013-07-26 19:07:00,2013-07-26 19:07:00,2013-07-26,2013-07-25,2013-07-01
52,354,62516JM,COM,NY,Delivery Truck,46-Double Parking (Com Plate),103,Queens,87-80,168th pl,2013-07-19 14:30:00,2013-07-19 14:30:00,2013-07-19,2013-07-18,2013-07-01
53,355,36095JG,COM,NY,Delivery Truck,19-No Stand (bus stop),103,Queens,160-05,jamaica ave,2013-07-30 12:50:00,2013-07-30 12:50:00,2013-07-30,2013-07-25,2013-07-01


In [660]:
#Which streets were the ones that had the most violations for passenger vehicles in Queens county
queens_name_street_pas = queens_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_name_street_pas['percentage'] = (queens_name_street_pas['plate_id'] / 4545072)*100
queens_name_street_pas

,street_name,plate_id,percentage
0,queens blvd,149738,3.294513
1,jamaica ave,115474,2.540642
2,37th ave,90130,1.983027
3,roosevelt ave,73805,1.623847
4,main st,63336,1.393509
5,broadway,60505,1.331222
6,steinway st,58494,1.286976
7,northern blvd,57857,1.272961
8,austin st,50155,1.103503
9,liberty ave,46057,1.013339


In [661]:
fig = px.bar(queens_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in Queens")
fig.show()

The most number of violations in the Queens county that has made passenger cars are on Queens Boulevard - 3.29% of the total violations. Other streets are: Jamaica Avenue - 2.54%, 37th Avenue - 1.98%, Roosevelt Avenue - 1.62% and Main Streen - 1.39%

In [662]:
#Which streets were the ones that had the most violations for commercial vehicles in Queens county
queens_name_street_com = queens_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_name_street_com['percentage'] = (queens_name_street_com['plate_id'] / 788373)*100
queens_name_street_com

,street_name,plate_id,percentage
0,queens blvd,35630,4.519434
1,jamaica ave,30100,3.817990
2,roosevelt ave,28507,3.615928
3,broadway,25502,3.234763
4,37th ave,21508,2.728150
5,main st,20652,2.619572
6,northern blvd,20395,2.586973
7,steinway st,14640,1.856989
8,austin st,11622,1.474175
9,30th ave,11341,1.438532


In [663]:
fig = px.bar(queens_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for commercial vehicles in Queens")
fig.show()

Most popular streets for comercial vehicles violations are: Queens Boulevard - 4.52% of the total number violations, Jamaica Avenue - 3.82%, Roosevelt Avenue - 3.62%, Broadway - 3.23% and 37th Avenue - 2.73%. 

4 out of 5 streets are the same streets that are most popular for passenger car violations.

In [664]:
#Number of daily violations for passenger vehicles
queens_daily_pas = queens_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
queens_daily_pas.head()

,day,plate_id
0,2013-07-01,3
1,2013-07-02,5
2,2013-07-03,7
3,2013-07-04,7
4,2013-07-05,1


In [665]:
fig = px.line(queens_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall trand is stable, but there was a significant reduction in the number of violations between May and July 2016.

In [666]:
#Number of weekly violations for passenger vehicles
queens_weekly_pas = queens_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
queens_weekly_pas.head()

,week,plate_id
0,2013-06-27,15
1,2013-07-04,31
2,2013-07-11,258
3,2013-07-18,1157
4,2013-07-25,12291


In [667]:
fig = px.line(queens_weekly_pas,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly number of passenger type vehicle violations in Queens is pretty much consistent, apart from the decrease at the end of each year and subsequent rise to average level. The only significant drop which is difficult to explain is from March 2016 to middle of June 2016.

In [668]:
#Number of monthly violations for passenger vehicles
queens_monthly_pas = queens_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
queens_monthly_pas.head()

,month,plate_id
0,2013-07-01,13752
1,2013-08-01,103051
2,2013-09-01,104766
3,2013-10-01,120582
4,2013-11-01,113281


In [669]:
fig = px.line(queens_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend shows that there is a overall similar amount of violations made py passenger type vehicles. There is a surprising drop in May-June 2016. 

In [670]:
#Number of daily violations for commercial vehicles
queens_daily_com = queens_ny_com.groupby(['day'])['plate_id'].count().reset_index()
queens_daily_com.head()

,day,plate_id
0,2013-07-01,1
1,2013-07-03,2
2,2013-07-09,3
3,2013-07-10,1
4,2013-07-11,4


In [671]:
fig = px.line(queens_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars daily number of violations')
fig.show()

Daily trend of violations made by commercial type of vehicles in Queens show similar picture as in other counties - number of violations by commercial type vehicles decrease on the weekends and at the end of each year. Overall the number of violations from 2016 is lower than it was before.

In [672]:
#Number of weekly violations for commercial vehicles
queens_weekly_com = queens_ny_com.groupby(['week'])['plate_id'].count().reset_index()
queens_weekly_com.head()

,week,plate_id
0,2013-06-27,3
1,2013-07-04,4
2,2013-07-11,69
3,2013-07-18,363
4,2013-07-25,2757


In [673]:
fig = px.line(queens_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars weekly number of violations')
fig.show()

Weekly trend shows that there is a decrease in violations in Queens by commercial type of vehicle at the end of each year and that there is a lower on average number of violations in 2016 onwards comparing to previous years. 

In [674]:
#Number of monthly violations for commercial vehicles
queens_monthly_com = queens_ny_com.groupby(['month'])['plate_id'].count().reset_index()
queens_monthly_com.head()

,month,plate_id
0,2013-07-01,3196
1,2013-08-01,19871
2,2013-09-01,20624
3,2013-10-01,24028
4,2013-11-01,20047


In [675]:
fig = px.line(queens_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars monthly number of violations')
fig.show()

Monthly trend of violations made by commercial type of vehicles in Queens shows that at the end of each year there is the lowest amount of violations and that the number of monthly violations in 2016 onwards has decreased, comparing to previous years. 

In [676]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in Queens county
queens_violations_precincts_pas = queens_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_violations_precincts_pas['percentage'] = (queens_violations_precincts_pas['plate_id'] / 4715404)*100
queens_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,114,708523,15.025711
1,109,553759,11.743617
2,112,421814,8.945448
3,115,394125,8.358245
4,103,381576,8.092117
5,108,327057,6.935927
6,107,301938,6.403227
7,104,296937,6.297170
8,102,290093,6.152029
9,110,286944,6.085247


In [677]:
fig = px.bar(queens_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Queens that gave the most number of tickets for passenger type vehicles")
fig.show()

Precincts that gave the most number of tickets to the passenger type vehicles: 114th precinct - 15.02% of total tickets, 109th precinct - 11.74%, 112th precinct - 8.94% and 115th precinct - 8.36%

In [678]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in Queens county
queens_violations_precincts_com = queens_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
queens_violations_precincts_com['percentage'] = (queens_violations_precincts_com['plate_id'] / 914308)*100
queens_violations_precincts_com

,violation_precinct,plate_id,percentage
0,109,146792,16.054984
1,114,133149,14.562817
2,108,101858,11.140447
3,112,92353,10.100863
4,115,90381,9.885181
5,110,64954,7.104171
6,103,63687,6.965596
7,104,60240,6.588589
8,102,46596,5.096313
9,107,30847,3.373808


In [679]:
fig = px.bar(queens_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Queens that gave the most number of tickets for commercial type vehicles")
fig.show()

Precincts that gave the most number of tickets to the commercial type vehicles: 109th precints - 16.05% of total tickets, 114th precinct - 14.56%, 108th precinct - 11.14% and 112th precinct - 10.1%

# BRONX

In [680]:
#Filter the main dataframe to contain only data related to the Bronx county
bronx = data[data['violation_county'] == 'Bronx'].reset_index()
bronx.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,81,GEX5296,PAS,NY,Sedan,14-No Standing,44,Bronx,1130,grand concourse,2013-07-26 11:50:00,2013-07-26 11:50:00,2013-07-26,2013-07-25,2013-07-01
1,82,PQU5978,PAS,GA,Sedan,17-No Stand (exc auth veh),44,Bronx,1125,grand concourse,2013-07-11 07:14:00,2013-07-11 07:14:00,2013-07-11,2013-07-11,2013-07-01
2,83,867ZFX,PAS,CT,Sedan,17-No Stand (exc auth veh),44,Bronx,118,garnd concourse,2013-07-15 07:11:00,2013-07-15 07:11:00,2013-07-15,2013-07-11,2013-07-01
3,84,XK558L,PAS,NJ,Pickup,17-No Stand (exc auth veh),44,Bronx,1125,grand concourse,2013-07-25 09:30:00,2013-07-25 09:30:00,2013-07-25,2013-07-25,2013-07-01
4,109,STN2382,PAS,NC,Suburban,40-Fire Hydrant,44,Bronx,NaN,n165th st,2013-08-03 01:57:00,2013-08-03 01:57:00,2013-08-03,2013-08-01,2013-08-01


In [681]:
#Which registration state is the most popular in Bronx county
bronx_reg_state = bronx.groupby('registration_state')['plate_id'].count().reset_index()
bronx_reg_state.head()

,registration_state,plate_id
0,99,17009
1,AB,73
2,AK,383
3,AL,2828
4,AR,1351


In [682]:
fig = px.bar(bronx_reg_state, x="registration_state", y="plate_id", labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             }, title="Number of violations in Bronx by vehicle registration state")
fig.show()

78.64% of the violations came from locals from NY. Another 6.6% from NJ.

In [683]:
#Filter dataframe to contain only registration plates from NY
bronx_ny = bronx[bronx['registration_state'] == 'NY']

In [684]:
#Which plate types are the most popular in Bronx county
bronx_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,PAS,2485001
1,COM,379546
2,OMT,141459
3,SRF,35987
4,OMS,32411
5,999,10413
6,IRP,8906
7,ORG,6280
8,TRC,5384
9,SPO,3565


Passener vehicles ammount to 79.26% of the total violations in Bronx made by NY locals. Commercial vehicles amount to another 12.1%

In [685]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
bronx_ny_pas = bronx_ny[bronx_ny['plate_type'] == 'PAS']
bronx_ny_pas.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,81,GEX5296,PAS,NY,Sedan,14-No Standing,44,Bronx,1130,grand concourse,2013-07-26 11:50:00,2013-07-26 11:50:00,2013-07-26,2013-07-25,2013-07-01
6,115,X332JF,PAS,NY,Suburban,14-No Standing,45,Bronx,2,city is ave,2013-08-02 19:35:00,2013-08-02 19:35:00,2013-08-02,2013-08-01,2013-08-01
7,137,GEX4415,PAS,NY,Sedan,14-No Standing,43,Bronx,3,soundvie ave,2013-08-18 20:14:00,2013-08-18 20:14:00,2013-08-18,2013-08-15,2013-08-01
8,158,GHN2894,PAS,NY,Suburban,14-No Standing,45,Bronx,492,hutchinson river parkw,2013-08-18 18:41:00,2013-08-18 18:41:00,2013-08-18,2013-08-15,2013-08-01
10,261,HX54671,PAS,NY,Sedan,14-No Standing,43,Bronx,3,soundview,2013-07-24 16:55:00,2013-07-24 16:55:00,2013-07-24,2013-07-18,2013-07-01


In [686]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
bronx_ny_com = bronx_ny[bronx_ny['plate_type'] == 'COM']
bronx_ny_com.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
5,114,45940MD,COM,NY,Van,78-Nighttime PKG on Res Street,45,Bronx,416,kirby st,2013-08-01 23:45:00,2013-08-01 23:45:00,2013-08-01,2013-08-01,2013-08-01
9,161,48686MA,COM,NY,Van,21-No Parking (street clean),41,Bronx,770,garrison ave,2013-08-16 01:40:00,2013-08-16 01:40:00,2013-08-16,2013-08-15,2013-08-01
73,1011,67325JY,COM,NY,Delivery Truck,85-Storage-3 hour Commercial,47,Bronx,NaN,co 233rd st,2013-07-25 01:30:00,2013-07-25 01:30:00,2013-07-25,2013-07-25,2013-07-01
113,1109,46353MA,COM,NY,Sedan,21-No Parking (street clean),50,Bronx,5913,broadway,2013-08-17 07:40:00,2013-08-17 07:40:00,2013-08-17,2013-08-15,2013-08-01
154,1375,87925JW,COM,NY,Van,78-Nighttime PKG on Res Street,45,Bronx,NaN,co bruckner blvd,2013-07-25 02:45:00,2013-07-25 02:45:00,2013-07-25,2013-07-25,2013-07-01


In [687]:
#Which violations are the most popular for vehicles with passenger plates in Bronx county
bronx_violation_pas = bronx_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_violation_pas['percentage'] = (bronx_violation_pas['plate_id'] / 2485001)*100
bronx_violation_pas

,violation_description,plate_id,percentage
0,21-No Parking (street clean),578916,23.296409
1,38-Failure to Display Muni Rec,325556,13.100840
2,20A-No Parking (Non-COM),266359,10.718668
3,20-No Parking (Com Plate),217885,8.768005
4,37-Expired Muni Meter,197571,7.950540
5,46-Double Parking (Com Plate),185068,7.447401
6,40-Fire Hydrant,161063,6.481406
7,70-Reg. Sticker Missing (NYS),118056,4.750743
8,14-No Standing,111585,4.490340
9,19-No Stand (bus stop),69236,2.786156


In [688]:
fig = px.bar(bronx_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Bronx county made by vehicles with passenger plates")
fig.show()

For passengers, 23.30% are for no parking during street cleaning hours, 13.10% are for failure to diplay receipt, 10.72% and 8.77% are no general parking for non commercial vehicle and for vehicle with commercial plate respectively and 7.95% are for expired municipal meter. 

In [689]:
#Which violations are the most popular for vehicles with commercial plates in Bronx county
bronx_violation_com = bronx_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_violation_com['percentage'] = (bronx_violation_com['plate_id'] / 379546)*100
bronx_violation_com

,violation_description,plate_id,percentage
0,46-Double Parking (Com Plate),74657,19.670080
1,38-Failure to Display Muni Rec,71101,18.733171
2,21-No Parking (street clean),30275,7.976635
3,19-No Stand (bus stop),27243,7.177786
4,48-Bike Lane,26302,6.929858
5,14-No Standing,26116,6.880852
6,78-Nighttime PKG on Res Street,23217,6.117045
7,40-Fire Hydrant,22379,5.896255
8,37-Expired Muni Meter,18731,4.935107
9,20A-No Parking (Non-COM),12226,3.221217


In [690]:
fig = px.bar(bronx_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Bronx county made by vehicles with commercial plates")
fig.show()

For comercial vehicles, 19.67% are for double parking violation, 18.73% are for failure to display receipt, 7.98% are for no parking during street cleaning hours, 7.18% are for no stand at bus stop violation and 6.93% are for parking at bike lane.

In [691]:
#Which body types are the most popular out of all passenger vehicles in Bronx county
bronx_vehicle_type_pas = bronx_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_vehicle_type_pas['percentage'] = (bronx_vehicle_type_pas['plate_id'] / 2485001)*100
bronx_vehicle_type_pas

,vehicle_body_type,plate_id,percentage
0,Sedan,1235322,49.711127
1,Suburban,1153394,46.414227
2,Van,41025,1.650905
3,Pickup,34069,1.370985
4,Convertible,10866,0.437263
5,Others,8321,0.334849
6,Delivery Truck,657,0.026439
7,Taxi,444,0.017867
8,Bus,280,0.011268
9,Utility,278,0.011187


In [692]:
fig = px.bar(bronx_vehicle_type_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Passenger type vehicle body types that had the most violations in Bronx")
fig.show()

Sedans and Suburbans are basically the only vehicle body types of passenger cars in Bronx - 49.71% and 46.41%

In [693]:
#Which body types are the most popular out of all commercial vehicles in Bronx county
bronx_vehicle_type_com = bronx_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_vehicle_type_com['percentage'] = (bronx_vehicle_type_com['plate_id'] / 379546)*100
bronx_vehicle_type_com

,vehicle_body_type,plate_id,percentage
0,Van,250888,66.102133
1,Delivery Truck,68820,18.132190
2,Pickup,19907,5.244951
3,Utility,12516,3.297624
4,Suburban,8374,2.206320
5,Refrigerator Trailer,7457,1.964716
6,Others,4505,1.186944
7,Sedan,2111,0.556191
8,Trailer,1587,0.418131
9,Tractor,1431,0.377029


In [694]:
fig = px.bar(bronx_vehicle_type_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in Bronx")
fig.show()

With regards to the commercial vehicle, Vans had the highest amount of violations - 66.10% following by Delivery trucks - 18.13% and Pickups - 5.24%

In [695]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
bronx_ny_pas_sub_sed = bronx_ny_pas[(bronx_ny_pas['vehicle_body_type'] == 'Sedan') | (bronx_ny_pas['vehicle_body_type'] == 'Suburban')]
bronx_ny_pas_sub_sed.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,81,GEX5296,PAS,NY,Sedan,14-No Standing,44,Bronx,1130,grand concourse,2013-07-26 11:50:00,2013-07-26 11:50:00,2013-07-26,2013-07-25,2013-07-01
6,115,X332JF,PAS,NY,Suburban,14-No Standing,45,Bronx,2,city is ave,2013-08-02 19:35:00,2013-08-02 19:35:00,2013-08-02,2013-08-01,2013-08-01
7,137,GEX4415,PAS,NY,Sedan,14-No Standing,43,Bronx,3,soundvie ave,2013-08-18 20:14:00,2013-08-18 20:14:00,2013-08-18,2013-08-15,2013-08-01
8,158,GHN2894,PAS,NY,Suburban,14-No Standing,45,Bronx,492,hutchinson river parkw,2013-08-18 18:41:00,2013-08-18 18:41:00,2013-08-18,2013-08-15,2013-08-01
10,261,HX54671,PAS,NY,Sedan,14-No Standing,43,Bronx,3,soundview,2013-07-24 16:55:00,2013-07-24 16:55:00,2013-07-24,2013-07-18,2013-07-01


In [696]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
bronx_ny_pas_van_delv_pick = bronx_ny_com[(bronx_ny_com['vehicle_body_type'] == 'Van') | (bronx_ny_com['vehicle_body_type'] == 'Delivery Truck') | (bronx_ny_com['vehicle_body_type'] == 'Pickup')]
bronx_ny_pas_van_delv_pick.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
5,114,45940MD,COM,NY,Van,78-Nighttime PKG on Res Street,45,Bronx,416,kirby st,2013-08-01 23:45:00,2013-08-01 23:45:00,2013-08-01,2013-08-01,2013-08-01
9,161,48686MA,COM,NY,Van,21-No Parking (street clean),41,Bronx,770,garrison ave,2013-08-16 01:40:00,2013-08-16 01:40:00,2013-08-16,2013-08-15,2013-08-01
73,1011,67325JY,COM,NY,Delivery Truck,85-Storage-3 hour Commercial,47,Bronx,NaN,co 233rd st,2013-07-25 01:30:00,2013-07-25 01:30:00,2013-07-25,2013-07-25,2013-07-01
154,1375,87925JW,COM,NY,Van,78-Nighttime PKG on Res Street,45,Bronx,NaN,co bruckner blvd,2013-07-25 02:45:00,2013-07-25 02:45:00,2013-07-25,2013-07-25,2013-07-01
155,1385,90533MC,COM,NY,Delivery Truck,46-Double Parking (Com Plate),40,Bronx,2958,3rd ave,2013-07-23 13:10:00,2013-07-23 13:10:00,2013-07-23,2013-07-18,2013-07-01


In [697]:
#Which streets were the ones that had the most violations for passenger vehicles in Bronx county
bronx_name_street_pas = bronx_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_name_street_pas['percentage'] = (bronx_name_street_pas['plate_id'] / 2388716)*100
bronx_name_street_pas

,street_name,plate_id,percentage
0,white plains rd,84035,3.517999
1,tremont ave,63230,2.647029
2,chester ave,56363,2.359552
3,jerome ave,55811,2.336444
4,grand concourse,46856,1.961556
5,webster ave,30244,1.266120
6,morris ave,26309,1.101387
7,southern blvd,25582,1.070952
8,morris park ave,24996,1.046420
9,broadway,24397,1.021344


In [698]:
fig = px.bar(bronx_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in Bronx")
fig.show()

Most of the violations in Bronx for passenger vehicles happened on White Plains Road - 3.52%. Following that streets with most violations from passenger vehicles are E Tremont Avenue - 2.65%, Chester Avenue - 2.36%, Jerome Avenue - 2.34% and Grand Concourse - 1.96%

In [699]:
#Which streets were the ones that had the most violations for commercial vehicles in Bronx county
bronx_name_street_com = bronx_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_name_street_com['percentage'] = (bronx_name_street_com['plate_id'] / 339615)*100
bronx_name_street_com

,street_name,plate_id,percentage
0,grand concourse,22080,6.501480
1,white plains rd,13584,3.999823
2,tremont ave,11737,3.455972
3,chester ave,10616,3.125893
4,jerome ave,10358,3.049924
5,webster ave,8279,2.437760
6,fordham rd,7658,2.254906
7,southern blvd,6140,1.807930
8,149th st,6003,1.767590
9,university ave,5293,1.558530


In [700]:
fig = px.bar(bronx_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for commercial vehicles in Bronx")
fig.show()

Most of the violations in Bronx for commercial vehicles happened on the Grand Concourse - 6.50%. Following that streets with most violations from passenger vehicles are White Plains Road - 4%, E Tremont Avenue - 3.46%, Chester Avenue - 3.13% and Jerome Avenue - 3.05% - same streets as for passenger vehicles.

In [701]:
#Number of daily violations for passenger vehicles
bronx_daily_pas = bronx_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
bronx_daily_pas.head()

,day,plate_id
0,2013-07-01,9
1,2013-07-02,4
2,2013-07-03,2
3,2013-07-04,1
4,2013-07-05,8


In [702]:
fig = px.line(bronx_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall trand is stable, but there was a significant reduction in the number of violations between May and July 2016. Also there are massive peaks in the number of violations on the end of November of each year.

In [703]:
#Number of weekly violations for passenger vehicles
bronx_weekly_pas = bronx_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
bronx_weekly_pas.head()

,week,plate_id
0,2013-06-27,15
1,2013-07-04,27
2,2013-07-11,76
3,2013-07-18,1216
4,2013-07-25,6699


In [704]:
fig = px.line(bronx_weekly_pas,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly number of passenger type vehicle violations in Bronx is pretty much consistent, apart from the decrease at the end of each year and subsequent rise to average level. The only significant drop which is difficult to explain is from March 2016 to middle of June 2016.

In [705]:
#Number of monthly violations for passenger vehicles
bronx_monthly_pas = bronx_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
bronx_monthly_pas.head()

,month,plate_id
0,2013-07-01,8033
1,2013-08-01,49497
2,2013-09-01,52224
3,2013-10-01,57574
4,2013-11-01,55033


In [706]:
fig = px.line(bronx_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend shows that there is a overall similar amount of violations made py passenger type vehicles in Bronx. There is a surprising drop in May-June 2016. 

In [707]:
#Number of daily violations for commercial vehicles
bronx_daily_com = bronx_ny_com.groupby(['day'])['plate_id'].count().reset_index()
bronx_daily_com.head()

,day,plate_id
0,2013-07-01,1
1,2013-07-10,2
2,2013-07-17,6
3,2013-07-18,18
4,2013-07-19,14


In [708]:
fig = px.line(bronx_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars daily number of violations')
fig.show()

Daily trend of violations made by commercial type of vehicles in Bronx show similar picture as in other counties - number of violations by commercial type vehicles decrease on the weekends and at the end of each year. Overall the number of violations from 2016 is lower than it was before.

In [709]:
#Number of weekly violations for commercial vehicles
bronx_weekly_com = bronx_ny_com.groupby(['week'])['plate_id'].count().reset_index()
bronx_weekly_com.head()

,week,plate_id
0,2013-06-27,1
1,2013-07-04,2
2,2013-07-11,6
3,2013-07-18,132
4,2013-07-25,1085


In [710]:
fig = px.line(bronx_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars weekly number of violations')
fig.show()

Weekly trend shows that there is a decrease in violations in Bronx by commercial type of vehicle at the end of each year and that there is a lower on average number of violations in 2016 onwards comparing to previous years. 

In [711]:
#Number of monthly violations for commercial vehicles
bronx_monthly_com = bronx_ny_com.groupby(['month'])['plate_id'].count().reset_index()
bronx_monthly_com.head()

,month,plate_id
0,2013-07-01,1226
1,2013-08-01,6991
2,2013-09-01,7234
3,2013-10-01,8243
4,2013-11-01,6478


In [712]:
fig = px.line(bronx_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial cars monthly number of violations')
fig.show()

Monthly trend of violations made by commercial type of vehicles in Queens shows that at the end of each year there is the lowest amount of violations and that the number of monthly violations in 2016 onwards has decreased, comparing to previous years.

In [713]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in Bronx county
bronx_violations_precincts_pas = bronx_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_violations_precincts_pas['percentage'] = (bronx_violations_precincts_pas['plate_id'] / 2485001)*100
bronx_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,52,339993,13.681805
1,49,282793,11.379995
2,43,267799,10.776615
3,46,248931,10.017340
4,47,239185,9.625147
5,45,217414,8.749051
6,48,186618,7.509776
7,44,186298,7.496898
8,50,183603,7.388448
9,40,159510,6.418911


In [714]:
fig = px.bar(bronx_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Bronx that gave the most number of tickets for passenger type vehicles")
fig.show()

Precincts that gave the most number of tickets to the passenger type vehicles: 52nd precinct - 13.68% of total tickets, 49th precinct - 11.38%, 43rd precinct - 10.78% and 46th precinct - 10.02%

In [715]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in Bronx county
bronx_violations_precincts_com = bronx_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
bronx_violations_precincts_com['percentage'] = (bronx_violations_precincts_com['plate_id'] / 379546)*100
bronx_violations_precincts_com

,violation_precinct,plate_id,percentage
0,52,57533,15.158373
1,46,52192,13.751166
2,43,38653,10.184009
3,49,36423,9.596465
4,40,34044,8.969664
5,44,28961,7.630432
6,48,27616,7.276061
7,45,26774,7.054217
8,47,25834,6.806553
9,50,21977,5.790339


In [716]:
fig = px.bar(bronx_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             },title="Precincts in Bronx that gave the most number of tickets for commercial type vehicles")
fig.show()

Precincts that gave the most number of tickets to the comercial type vehicles: 52nd precinct - 15.16% of total tickets, 46th precinct - 13.75%, 43rd precinct - 10.18% and 49th precinct - 9.60%

# STATEN ISLAND

In [717]:
#Filter the main dataframe to contain only data related to the Staten Island county
staten = data[data['violation_county'] == 'Staten Island'].reset_index()
staten.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
0,35,G24BLB,PAS,NJ,Sedan,40-Fire Hydrant,120,Staten Island,257,osgood ave,2013-07-19 11:35:00,2013-07-19 11:35:00,2013-07-19,2013-07-18,2013-07-01
1,36,GER8956,SRF,NY,Sedan,14-No Standing,120,Staten Island,265,victory blvd,2013-07-26 18:37:00,2013-07-26 18:37:00,2013-07-26,2013-07-25,2013-07-01
2,102,GBN7123,PAS,NY,Suburban,24-No Parking (exc auth veh),122,Staten Island,NaN,great kills park,2013-07-30 12:55:00,2013-07-30 12:55:00,2013-07-30,2013-07-25,2013-07-01
3,103,GDE3096,PAS,NY,Others,24-No Parking (exc auth veh),122,Staten Island,NaN,great kill park boat,2013-07-24 20:27:00,2013-07-24 20:27:00,2013-07-24,2013-07-18,2013-07-01
4,145,GAB6661,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,NaN,crescententent beach park,2013-08-11 15:25:00,2013-08-11 15:25:00,2013-08-11,2013-08-08,2013-08-01


In [718]:
#Which registration state is the most popular in Staten Island county
staten_reg_state = staten.groupby('registration_state')['plate_id'].count().reset_index()
staten_reg_state.head()

,registration_state,plate_id
0,99,1196
1,AB,5
2,AK,42
3,AL,147
4,AR,85


In [719]:
fig = px.bar(staten_reg_state, x="registration_state", y="plate_id", labels={
                     "registration_state" : "Registration State",
                     "plate_id" : "Number of Violations"
             }, title="Number of violations in Staten Island by vehicle registration state")
fig.show()

88.9% of the violations came from locals from NY

In [720]:
#Filter dataframe to contain only registration plates from NY
staten_ny = staten[staten['registration_state'] == 'NY']

In [721]:
#Which plate types are the most popular in NYC
staten_ny.groupby(['plate_type'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)

,plate_type,plate_id
0,PAS,319339
1,COM,16287
2,SRF,4090
3,OMS,1630
4,OMT,1515
5,999,1083
6,ORG,848
7,SPO,371
8,RGL,243
9,TOW,223


Passener vehicles ammount to 91.9% of the total violations in Staten island made by NY locals. Commercial vehicles amount to another 4.69%

In [722]:
#Further filtering dataframe
#Filtering dataframe to contain only vehicles with passenger plate types
staten_ny_pas = staten_ny[staten_ny['plate_type'] == 'PAS']
staten_ny_pas.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
2,102,GBN7123,PAS,NY,Suburban,24-No Parking (exc auth veh),122,Staten Island,NaN,great kills park,2013-07-30 12:55:00,2013-07-30 12:55:00,2013-07-30,2013-07-25,2013-07-01
3,103,GDE3096,PAS,NY,Others,24-No Parking (exc auth veh),122,Staten Island,NaN,great kill park boat,2013-07-24 20:27:00,2013-07-24 20:27:00,2013-07-24,2013-07-18,2013-07-01
4,145,GAB6661,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,NaN,crescententent beach park,2013-08-11 15:25:00,2013-08-11 15:25:00,2013-08-11,2013-08-08,2013-08-01
7,148,GGS6051,PAS,NY,Sedan,24-No Parking (exc auth veh),122,Staten Island,NaN,dolphin circle,2013-07-27 15:44:00,2013-07-27 15:44:00,2013-07-27,2013-07-25,2013-07-01
10,151,GAW6458,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,NaN,south beach,2013-08-17 12:40:00,2013-08-17 12:40:00,2013-08-17,2013-08-15,2013-08-01


In [723]:
#Filtering the same dataframe to contain only vehicles with commercial plate types
staten_ny_com = staten_ny[staten_ny['plate_type'] == 'COM']
staten_ny_com.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
18,2071,48454JX,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,NaN,co richmond ave,2013-07-27 22:50:00,2013-07-27 22:50:00,2013-07-27,2013-07-25,2013-07-01
53,4966,83930J2,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,149,bidwell ave,2013-08-09 00:15:00,2013-08-09 00:15:00,2013-08-09,2013-08-08,2013-08-01
58,4971,52858JY,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,1210,richmond ave,2013-07-27 22:50:00,2013-07-27 22:50:00,2013-07-27,2013-07-25,2013-07-01
59,4972,77477JZ,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,193,south ave,2013-08-03 23:00:00,2013-08-03 23:00:00,2013-08-03,2013-08-01,2013-08-01
62,4975,52013JX,COM,NY,Trailer,78-Nighttime PKG on Res Street,121,Staten Island,47-69,soren st,2013-07-21 22:15:00,2013-07-21 22:15:00,2013-07-21,2013-07-18,2013-07-01


In [724]:
#Which violations are the most popular for vehicles with passenger plates in Staten Island county
staten_violation_pas = staten_ny_pas.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_violation_pas['percentage'] = (staten_violation_pas['plate_id'] / 319339)*100
staten_violation_pas

,violation_description,plate_id,percentage
0,20A-No Parking (Non-COM),113130,35.426302
1,38-Failure to Display Muni Rec,46951,14.702557
2,70-Reg. Sticker Missing (NYS),40628,12.722530
3,14-No Standing,26070,8.163738
4,20-No Parking (Com Plate),19995,6.261371
5,37-Expired Muni Meter,18723,5.863048
6,40-Fire Hydrant,16867,5.281848
7,19-No Stand (bus stop),5451,1.706963
8,27-No Parking (exc handicap),4849,1.518449
9,46-Double Parking (Com Plate),3346,1.047789


In [725]:
fig = px.bar(staten_violation_pas, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             },title="Popular violations in Staten Island county made by vehicles with passenger plates")
fig.show()

For passengers, 35.43% are for no parking violation, 14.70% are for failure to diplay receipt, 12.72% are for missing registration sticker, 8.16% are for no standing violation and 6.26% are for no parking with commercial plate violation.

In [726]:
#Which violations are the most popular for vehicles with commercial plates in Staten Island county
staten_violation_com = staten_ny_com.groupby(['violation_description'])['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_violation_com['percentage'] = (staten_violation_com['plate_id'] / 16287)*100
staten_violation_com

,violation_description,plate_id,percentage
0,78-Nighttime PKG on Res Street,4582,28.132867
1,38-Failure to Display Muni Rec,3337,20.488733
2,14-No Standing,2002,12.292012
3,20A-No Parking (Non-COM),1815,11.143857
4,19-No Stand (bus stop),944,5.796034
5,40-Fire Hydrant,769,4.721557
6,70-Reg. Sticker Missing (NYS),527,3.235709
7,37-Expired Muni Meter,468,2.873457
8,82-Unaltered Commerc Vehicle,426,2.615583
9,46-Double Parking (Com Plate),290,1.780561


In [727]:
fig = px.bar(staten_violation_com, x="violation_description", y="plate_id", labels={
                     "violation_description" : "Violation Description",
                     "plate_id" : "Number of Violations"
             }, title="Popular violations in Staten Island county made by vehicles with commercial plates")
fig.show()

For comercial vehicles, 28.13% are for nighttime parking on residential street violation, 20.49% are for failure to display municipal receipt, 12.29% are for for no standing violation, 11.14% are for no parking violation and 5.8% are for no standing at bus stop violation.

In [728]:
#Which body types are the most popular out of all passenger vehicles in Staten Island county
staten_plate_pas = staten_ny_pas.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_plate_pas['percentage'] = (staten_plate_pas['plate_id'] / 319339)*100
staten_plate_pas

,vehicle_body_type,plate_id,percentage
0,Sedan,169049,52.937161
1,Suburban,135131,42.315846
2,Pickup,9366,2.932933
3,Convertible,2441,0.764391
4,Van,2145,0.671700
5,Others,1020,0.319410
6,Delivery Truck,55,0.017223
7,Trailer,45,0.014092
8,Bus,44,0.013778
9,Taxi,13,0.004071


In [729]:
fig = px.bar(staten_plate_pas, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             },title="Passenger type vehicle body types that had the most violations in Staten Island")
fig.show()

By far vehicle body types for passenger cars with most violations are Sedans of various types - 52.94% and Suburbans - 42.32%

In [730]:
#Which body types are the most popular out of all commercial vehicles in Staten Island county
staten_plate_com = staten_ny_com.groupby('vehicle_body_type')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_plate_com['percentage'] = (staten_plate_com['plate_id'] / 16287)*100
staten_plate_com

,vehicle_body_type,plate_id,percentage
0,Van,10067,61.810033
1,Delivery Truck,2393,14.692700
2,Pickup,2006,12.316571
3,Suburban,648,3.978633
4,Utility,459,2.818199
5,Others,366,2.247191
6,Sedan,139,0.853441
7,Refrigerator Trailer,74,0.454350
8,Trailer,45,0.276294
9,Flat Bed Truck,37,0.227175


In [731]:
fig = px.bar(staten_plate_com, x="vehicle_body_type", y="plate_id", labels={
                     "vehicle_body_type" : "Vehicle Body Type",
                     "plate_id" : "Number of Violations"
             }, title="Commercial type vehicle body types that had the most violations in Staten Island")
fig.show()

With regards to the commercial vehicle, Vans had the highest amount of violations - 61.81% following by Delivery trucks - 14.69% and Pickups - 12.32%

In [732]:
#Further filtering the dataframe to contain only sedans and suburbans out of all passenger vehicles
staten_ny_pas_sub_sed = staten_ny_pas[(staten_ny_pas['vehicle_body_type'] == 'Sedan') | (staten_ny_pas['vehicle_body_type'] == 'Suburban')]
staten_ny_pas_sub_sed.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
2,102,GBN7123,PAS,NY,Suburban,24-No Parking (exc auth veh),122,Staten Island,NaN,great kills park,2013-07-30 12:55:00,2013-07-30 12:55:00,2013-07-30,2013-07-25,2013-07-01
4,145,GAB6661,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,NaN,crescententent beach park,2013-08-11 15:25:00,2013-08-11 15:25:00,2013-08-11,2013-08-08,2013-08-01
7,148,GGS6051,PAS,NY,Sedan,24-No Parking (exc auth veh),122,Staten Island,NaN,dolphin circle,2013-07-27 15:44:00,2013-07-27 15:44:00,2013-07-27,2013-07-25,2013-07-01
10,151,GAW6458,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,NaN,south beach,2013-08-17 12:40:00,2013-08-17 12:40:00,2013-08-17,2013-08-15,2013-08-01
17,1800,GDN8983,PAS,NY,Sedan,20-No Parking (Com Plate),122,Staten Island,460,briellavsi ny 10,2013-07-17 13:35:00,2013-07-17 13:35:00,2013-07-17,2013-07-11,2013-07-01


In [733]:
#Further filtering the dataframe to contain only vans, delivery trucks and pickups out of all commercial vehicles
staten_ny_pas_van_delv_pick = staten_ny_com[(staten_ny_com['vehicle_body_type'] == 'Van') | (staten_ny_com['vehicle_body_type'] == 'Delivery Truck') | (staten_ny_com['vehicle_body_type'] == 'Pickup')]
staten_ny_pas_van_delv_pick.head()

,index,plate_id,plate_type,registration_state,vehicle_body_type,violation_description,violation_precinct,violation_county,house_number,street_name,issue_datetime,time,day,week,month
18,2071,48454JX,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,NaN,co richmond ave,2013-07-27 22:50:00,2013-07-27 22:50:00,2013-07-27,2013-07-25,2013-07-01
53,4966,83930J2,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,149,bidwell ave,2013-08-09 00:15:00,2013-08-09 00:15:00,2013-08-09,2013-08-08,2013-08-01
58,4971,52858JY,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,1210,richmond ave,2013-07-27 22:50:00,2013-07-27 22:50:00,2013-07-27,2013-07-25,2013-07-01
59,4972,77477JZ,COM,NY,Van,78-Nighttime PKG on Res Street,121,Staten Island,193,south ave,2013-08-03 23:00:00,2013-08-03 23:00:00,2013-08-03,2013-08-01,2013-08-01
72,10152,83163MD,COM,NY,Delivery Truck,78-Nighttime PKG on Res Street,120,Staten Island,31,nowood ave,2013-07-19 21:28:00,2013-07-19 21:28:00,2013-07-19,2013-07-18,2013-07-01


In [734]:
#Which streets were the ones that had the most violations for passenger vehicles in Staten Island county
staten_name_street_pas = staten_ny_pas_sub_sed.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_name_street_pas['percentage'] = (staten_name_street_pas['plate_id'] / 304180)*100
staten_name_street_pas

,street_name,plate_id,percentage
0,richmond ave,21196,6.968242
1,bay st,10589,3.481162
2,mason ave,9382,3.084358
3,bard ave,9061,2.978828
4,victory blvd,7941,2.610625
5,seaveew ave,7794,2.562299
6,new dorp ln,7379,2.425866
7,forest ave,6685,2.197712
8,st marks pl,6327,2.080018
9,central ave,4412,1.450457


In [735]:
fig = px.bar(staten_name_street_pas, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             },title="Streets with most number of violations for passenger vehicles in Staten Island")
fig.show()

Surprisingly, on Richmond Avenue there was the greatest amount of Violations of passenger vehicles on Staten Island - 6.97% of all passenger car violations. Other popular streets are: Bay Street - 3.48%, Mason Avenue - 3.08, Bard Avenue - 2.98% and Victory Boulevard - 2.61%

In [736]:
#Which streets were the ones that had the most violations for commercial vehicles in Staten Island county
staten_name_street_com = staten_ny_pas_van_delv_pick.groupby('street_name')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_name_street_com['percentage'] = (staten_name_street_com['plate_id'] / 14466)*100
staten_name_street_com.head(15)

,street_name,plate_id,percentage
0,forest ave,769,5.315913
1,bay st,632,4.368865
2,new dorp ln,589,4.071616
3,victory blvd,565,3.905710
4,port richmond ave,527,3.643025
5,water st,365,2.523158
6,st marks pl,316,2.184432
7,richmond ave,299,2.066916
8,hylan blvd,264,1.824969
9,bard ave,239,1.652150


In [737]:
fig = px.bar(staten_name_street_com, x="plate_id", y="street_name", labels={
                     "street_name" : "Street Name",
                     "plate_id" : "Number of Violations"
             }, title="Streets with most number of violations for commercial vehicles in Staten Island")
fig.show()

Majority of the commercial vehicles violations were on Forest Avenue - 5.32% of the total violations, on Bay Street - 4.37% and on New Dorp Lane - 4.07%, on Victory Boulevard - 3.91% and on Port Richmond Avenue - 3.64%. As all of the numbers are very low, there is no one specific place for lots of violations from commercial vehcles. 

In [738]:
#Number of daily violations for passenger vehicles
staten_daily_pas = staten_ny_pas.groupby(['day'])['plate_id'].count().reset_index()
staten_daily_pas.head()

,day,plate_id
0,2013-07-03,4
1,2013-07-05,1
2,2013-07-06,3
3,2013-07-07,3
4,2013-07-08,4


In [739]:
fig = px.line(staten_daily_pas,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars daily number of violations')
fig.show()

Graph shows that number of violations made by passenger cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall there were more daily violations in the 2014, then there was a decrease in 2015-2016 and from the second half of 2016 there was an increase in Violations. Furthermore, there was a significant reduction in the number of violations between May and July 2016.

In [740]:
#Number of weekly violations for passenger vehicles
staten_weekly_pas = staten_ny_pas.groupby(['week'])['plate_id'].count().reset_index()
staten_weekly_pas.head()

,week,plate_id
0,2013-06-27,4
1,2013-07-04,13
2,2013-07-11,108
3,2013-07-18,594
4,2013-07-25,1195


In [741]:
fig = px.line(staten_weekly_pas,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars weekly number of violations')
fig.show()

Weekly number of passenger type vehicle violations in Staten Island was increasing until the middle of 2014, then it was on the downwards trend, reaching a plateau in 2015-2016 and then the average daily number of violations rose again in 2017. And as in other counties, there is a significant drop in May-June 2016.

In [742]:
#Number of monthly violations for passenger vehicles
staten_monthly_pas = staten_ny_pas.groupby(['month'])['plate_id'].count().reset_index()
staten_monthly_pas.head()

,month,plate_id
0,2013-07-01,1914
1,2013-08-01,5103
2,2013-09-01,4526
3,2013-10-01,4959
4,2013-11-01,5400


In [743]:
fig = px.line(staten_monthly_pas,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Passenger cars monthly number of violations')
fig.show()

Monthly trend shows similar picture, increase in weekly violations - decrease - plateau and then increase again.

In [744]:
#Number of daily violations for commercial vehicles
staten_daily_com = staten_ny_com.groupby(['day'])['plate_id'].count().reset_index()
staten_daily_com.head()

,day,plate_id
0,2013-07-16,6
1,2013-07-17,4
2,2013-07-18,4
3,2013-07-19,5
4,2013-07-20,8


In [745]:
fig = px.line(staten_daily_com,
              x="day", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial vehicles daily number of violations')
fig.show()

Graph shows that number of violations made by commercial cars are peaking at the middle of the week and then, decrease and Saturday and Sunday are the days with lowers amount of violations. Overall there were more daily violations in the 2014, then there was a decrease in 2015-2016 and from the second half of 2016 there was an increase in Violations. Furthermore, there was a significant reduction in the number of violations between May and July 2016.

In [746]:
#Number of weekly violations for commercial vehicles
staten_weekly_com = staten_ny_com.groupby(['week'])['plate_id'].count().reset_index()
staten_weekly_com.head()

,week,plate_id
0,2013-07-11,10
1,2013-07-18,55
2,2013-07-25,56
3,2013-08-01,80
4,2013-08-08,66


In [747]:
fig = px.line(staten_weekly_com,
              x="week", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial vehicles weekly number of violations')
fig.show()

Weekly number of commercial type vehicle violations in Staten Island was increasing until the middle of 2014, then it was on the downwards trend, reaching a plateau in 2015-2016 and then the average daily number of violations rose again in 2017. And as in other counties, there is a significant drop in May-June 2016.

In [748]:
#Number of monthly violations for commercial vehicles
staten_monthly_com = staten_ny_com.groupby(['month'])['plate_id'].count().reset_index()
staten_monthly_com.head()

,month,plate_id
0,2013-07-01,121
1,2013-08-01,275
2,2013-09-01,255
3,2013-10-01,210
4,2013-11-01,265


In [749]:
fig = px.line(staten_monthly_com,
              x="month", 
              y="plate_id",
              labels={
                     "plate_id" : "Number of Violations"
             },
              title='Commercial vehicles monthly number of violations')
fig.show()

Monthly trend shows that there was a big number of monthly violations in 2014, then it decreased by 1.5 times approximately in 2015-middle of 2016 and after that in rose to the same level as in 2014.

In [750]:
#Which precincts were the ones that gave the most tickets for passenger vehicles in Staten Island county
staten_violations_precincts_pas = staten_ny_pas.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_violations_precincts_pas['percentage'] = (staten_violations_precincts_pas['plate_id'] / 319339)*100
staten_violations_precincts_pas

,violation_precinct,plate_id,percentage
0,120,122772,38.445664
1,122,99328,31.104250
2,121,67306,21.076661
3,123,29933,9.373424


In [751]:
fig = px.bar(staten_violations_precincts_pas, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             }, title="Precincts in Staten Island that gave the most number of tickets for passenger type vehicles")
fig.show()

Precincts that gave the most number of tickets to the passenger type vehicles: 120th precinct - 38.45% of total tickets, 122nd precinct - 31.10%, 121st precinct - 21.08% and 123th precinct - 9.37%

In [752]:
#Which precincts were the ones that gave the most tickets for commercial vehicles in Staten Island county
staten_violations_precincts_com = staten_ny_com.groupby('violation_precinct')['plate_id'].count().sort_values(ascending=False).reset_index().head(10)
staten_violations_precincts_com['percentage'] = (staten_violations_precincts_com['plate_id'] / 16287)*100
staten_violations_precincts_com

,violation_precinct,plate_id,percentage
0,120,6239,38.306625
1,122,4744,29.127525
2,121,3384,20.777307
3,123,1920,11.788543


In [753]:
fig = px.bar(staten_violations_precincts_com, x="violation_precinct", y="plate_id", labels={
                     "violation_precinct" : "Precinct Number",
                     "plate_id" : "Number of Violations"
             }, title="Precincts in Staten Island that gave the most number of tickets for commercial type vehicles")
fig.show()

Precincts that gave the most number of tickets to the commercial type vehicles: 120th precinct - 38.31% of total tickets, 122nd precinct - 29.13%, 121st precinct - 20.78% and 123th precinct - 11.79%